# Merge BnF DATA, DBpedia and Wikidata

In this notebook, we apply a method to merge three datasets (BnF, DBpedia and Wikidata)

* First, we drop duplicates of each datasets. 

* Secondly, we merge the three datasets  by removing duplicate data. To realise that, we will use the Linkage toolkit who calculate the proximity (by giving a score) between to string from three dataframes.

* Previously, we have to collect data about economists with SPARQL queries.

In [1]:
# Import libraries usefull

from SPARQLWrapper import SPARQLWrapper, SPARQLWrapper2, JSON, TURTLE, XML, RDFXML
import pprint
import csv
# from bs4 import BeautifulSoup

from collections import Counter
from operator import itemgetter
import pandas as pd


# Query economists and jurists on BnF Data

First step, we need datas about economists and jurists with theirs proprieties on 'BnF Data' so we realise a SPARQL query. We need proprieties to realise the merge between three datasets:
  * Birth date
  
  * Date of death
  
  * Place of Birth
  
  * Place of Death

In addition, we add the biographie to filter the population we need. 

In [47]:
query = """
PREFIX bio: <http://vocab.org/bio/0.1/>
PREFIX  egr:  <http://rdvocab.info/ElementsGr2/>
PREFIX  owl:  <http://www.w3.org/2002/07/owl#>
PREFIX  dbr:  <http://dbpedia.org/resource/>
PREFIX  xsd:  <http://www.w3.org/2001/XMLSchema#>
PREFIX  skos: <http://www.w3.org/2004/02/skos/core#>
PREFIX  foaf: <http://xmlns.com/foaf/0.1/>

SELECT DISTINCT  ?s ?name ?sName ?uri ?birthDate ?deathDate ?placeOfBirth ?placeOfDeath ?bio
WHERE
  {   { ?s  egr:biographicalInformation  ?bio ;
            egr:dateOfBirth       ?bd
        BIND(strbefore(strafter(str(?bd), "http://data.bnf.fr/date/"), "/") AS ?bd1)
        FILTER ( ( ( ( ( regex(?bio, "juriste", "i") || regex(?bio, "professeur de droit", "i") ) || regex(?bio, "docteur en droit", "i") ) || regex(?bio, "avocat", "i") ) || regex(?bio, "juge", "i") ) || regex(?bio, "magistrat", "i") )
        OPTIONAL
          { ?s  bio:birth ?birthDate }
        OPTIONAL
          { ?s  foaf:name  ?name }
        OPTIONAL
          { ?s  egr:placeOfBirth ?placeOfBirth}
        OPTIONAL
          { ?s  bio:death       ?deathDate
          }
        OPTIONAL
          { ?s  egr:placeOfDeath ?placeOfDeath}
        OPTIONAL
          { ?s  skos:prefLabel  ?sName }
        OPTIONAL
          { ?s  owl:sameAs  ?uri
            FILTER regex(?uri, "viaf.org", "i")
          }
      }
    UNION
      { ?s  egr:biographicalInformation  ?bio ;
            egr:dateOfBirth       ?bd
        BIND(strbefore(strafter(str(?bd), "http://data.bnf.fr/date/"), "/") AS ?bd1)
        FILTER ( ?birthDate > "1770" )
        FILTER ( ( ( regex(?bio, "économiste") || regex(?bio, "Economiste") ) || regex(?bio, "professeur d'économie", "i") ) || regex(?bio, "docteur en économie", "i") )
        OPTIONAL
          { ?s  bio:birth ?birthDate }
        OPTIONAL
          { ?s  foaf:name  ?name }
        OPTIONAL
          { ?s  egr:placeOfBirth ?placeOfBirth}
        OPTIONAL
          { ?s  egr:placeOfDeath ?placeOfDeath}
        OPTIONAL
          { ?s  bio:death       ?deathDate
          }
        OPTIONAL
          { ?s  skos:prefLabel  ?sName }
        OPTIONAL
          { ?s  owl:sameAs  ?uri1
            FILTER regex(?uri1, "viaf.org", "i")
            BIND(strbefore(str(?uri1), "http://viaf.org/viaf/") AS ?uri)
          }
      }
        FILTER ( ?bd1 > "1770" )
  }
ORDER BY DESC(?uri)
"""

In [48]:
sparql = SPARQLWrapper("https://data.bnf.fr/sparql") ##, returnFormat=RDFXML)  [LOCALHOST]

In [49]:
sparql.setQuery(query)
sparql.setReturnFormat(JSON)

In [50]:
try:
    rc_bnf = sparql.queryAndConvert()
except Exception as e:
    print(e)

In [51]:
# Number of rows in the result
len(rc_bnf['results']['bindings'])

11132

In [52]:
# Inspect the first hundred rows
i = 0
for l in rc_bnf['results']['bindings']:
    if i < 100:
        print(l)
        i += 1

{'s': {'type': 'uri', 'value': 'http://data.bnf.fr/ark:/12148/cb12981404c#about'}, 'name': {'type': 'literal', 'value': 'Léon Garnier'}, 'uri': {'type': 'uri', 'value': 'http://viaf.org/viaf/99996033'}, 'birthDate': {'type': 'literal', 'value': '1836-11-10'}, 'deathDate': {'type': 'literal', 'value': '1901-05-06'}, 'bio': {'type': 'literal', 'value': "Juriste. - Administrateur et homme de lettres. - En poste à la Préfecture de la Seine. - Frère de l'explorateur Francis Garnier (1839-1873)"}}
{'s': {'type': 'uri', 'value': 'http://data.bnf.fr/ark:/12148/cb13484444m#about'}, 'name': {'type': 'literal', 'value': 'Gaston de Pawlowski'}, 'uri': {'type': 'uri', 'value': 'http://viaf.org/viaf/9999219'}, 'birthDate': {'type': 'literal', 'value': '1874-06-14'}, 'deathDate': {'type': 'literal', 'value': '1933-02-02'}, 'placeOfBirth': {'type': 'literal', 'value': 'Joigny (Yonne)'}, 'placeOfDeath': {'type': 'literal', 'value': 'Paris'}, 'bio': {'type': 'literal', 'value': 'Docteur en droit. - Crit

In [53]:
result_bnf = []
for l in rc_bnf['results']['bindings']:
        try: 
            name = l['name']['value']
        except Exception as e:
            name = ''
        try: 
            sName = l['sName']['value']
        except Exception as e:
            sName = ''
        try: 
            uri = l['uri']['value']
        except Exception as e:
            uri = ''
        try: 
            bio = l['bio']['value']
        except Exception as e:
            bio = ''
        try: 
            birthDate = l['birthDate']['value']
        except Exception as e:
            birthDate = ''
        try: 
            deathDate = l['deathDate']['value']
        except Exception as e:
            deathDate = ''
        try: 
            placeOfBirth = l['placeOfBirth']['value']
        except Exception as e:
            placeOfBirth = ''
        try: 
            placeOfDeath = l['placeOfDeath']['value']
        except Exception as e:
            placeOfDeath = ''
        result_bnf.append([l['s']['value'], uri, name, sName, birthDate, deathDate, placeOfBirth,placeOfDeath, bio])        
# It creates a list, where it puts proprieties with their value.

In [54]:
print(len(result_bnf))
result_bnf[:5]

11132


[['http://data.bnf.fr/ark:/12148/cb12981404c#about',
  'http://viaf.org/viaf/99996033',
  'Léon Garnier',
  '',
  '1836-11-10',
  '1901-05-06',
  '',
  '',
  "Juriste. - Administrateur et homme de lettres. - En poste à la Préfecture de la Seine. - Frère de l'explorateur Francis Garnier (1839-1873)"],
 ['http://data.bnf.fr/ark:/12148/cb13484444m#about',
  'http://viaf.org/viaf/9999219',
  'Gaston de Pawlowski',
  '',
  '1874-06-14',
  '1933-02-02',
  'Joigny (Yonne)',
  'Paris',
  'Docteur en droit. - Critique littéraire et théâtral. - Rédacteur en chef de "Comoedia"'],
 ['http://data.bnf.fr/ark:/12148/cb134841632#about',
  'http://viaf.org/viaf/9999131',
  'Jean-Michel Berton',
  '',
  '1794-07-03',
  '1845-10-20',
  'Cahors (Lot)',
  '',
  'Écrivain politique, avocat à la Cour de cassation. - Fut fondateur et directeur de la "Revue poétique française et étrangère"'],
 ['http://data.bnf.fr/ark:/12148/cb13379520q#about',
  'http://viaf.org/viaf/9995247',
  'Emmanuel Mathieu',
  '',
  '1

# Query economists and jurists on DBpedia

The query is the same as BnF Data, but we add the nationalities, who they don't on BnF Data.

In [55]:
query_2= """
PREFIX  dbo:  <http://dbpedia.org/ontology/>
PREFIX  dbp:  <http://dbpedia.org/property/>
PREFIX  owl:  <http://www.w3.org/2002/07/owl#>
PREFIX  dbr:  <http://dbpedia.org/resource/>
PREFIX  xsd:  <http://www.w3.org/2001/XMLSchema#>
PREFIX  foaf: <http://xmlns.com/foaf/0.1/>

SELECT DISTINCT  ?s ?uri ?name ?birthDate ?deathDate ?abstract ?placeOfBirth ?placeOfDeath ?nationality
WHERE
  {   { ?s  a              dbo:Economist ;
            dbp:birthDate  ?birthDate
        FILTER ( xsd:date(?Birth_Date) > "1770-01-01"^^xsd:date )
        OPTIONAL
          { ?s  owl:sameAs  ?uri
            FILTER regex(?uri, "viaf", "i")
          }
        OPTIONAL
          { ?s  dbp:name  ?name }
        FILTER ( xsd:string(?name) != "Samuel Bowles" )
        OPTIONAL
          { ?s  dbp:birthPlace  ?bp }
        OPTIONAL
          { ?s  dbp:deathPlace  ?dp }
        OPTIONAL
          { ?s  dbp:deathDate  ?deathDate }
        OPTIONAL
          { ?s dbp:deathDate  ?deathDate }
        OPTIONAL
          { ?s  dbp:nationality  ?nationality1 }
          
      }
    UNION
      { ?s  ?propriety  dbr:Economist ;
            dbp:birthDate  ?birthDate

        FILTER ( xsd:date(?birthDate) > "1770-01-01"^^xsd:date )
        OPTIONAL
          { ?s  owl:sameAs  ?uri
            FILTER regex(?uri, "viaf", "i")
          }
        OPTIONAL
          { ?s  dbp:name  ?name
            FILTER ( xsd:string(?name) != "Samuel Bowles" )
          }
        OPTIONAL
          { ?s  dbp:birthPlace  ?bp }
        OPTIONAL
          { ?s  dbp:deathPlace  ?dp }
        OPTIONAL
          { ?s dbp:deathDate  ?deathDate }
        OPTIONAL
          { ?s  dbp:nationality  ?nationality1 }
      }
    UNION
      { ?s  ?p             dbr:Jurist ;
            dbp:birthDate  ?birthDate
        FILTER ( xsd:date(?birthDate) > "1770-01-01"^^xsd:date )
        OPTIONAL
          { ?s  dbp:name  ?name
            FILTER ( xsd:string(?name) != "Marcus Tullius Cicero" )
            FILTER ( xsd:string(?name) != "Samuel Bowles" )
          }
        OPTIONAL
          { ?s  owl:sameAs  ?uri
            FILTER regex(?uri, "viaf", "i")
          }
        OPTIONAL
          { ?s  dbp:birthPlace  ?bp }
        OPTIONAL
          { ?s  dbp:deathPlace  ?dp }
        OPTIONAL
          { ?s dbp:deathDate  ?deathDate }
        OPTIONAL
          { ?s  dbp:nationality  ?nationality1 }
      }
    UNION
      { ?s  ?p             dbr:Lawyer ;
            dbp:birthDate  ?birthDate
        FILTER ( xsd:date(?birthDate) > "1770-01-01"^^xsd:date )
        OPTIONAL
          { ?s  dbp:name  ?name
            FILTER ( xsd:string(?name) != "Marcus Tullius Cicero" )
            FILTER ( xsd:string(?name) != "Samuel Bowles" )
          }
        OPTIONAL
          { ?s  dbp:birthPlace  ?bp }
        OPTIONAL
          { ?s  dbp:deathPlace  ?dp }
        OPTIONAL
          { ?s dbp:deathDate  ?deathDate }
        OPTIONAL
          { ?s  dbp:nationality  ?nationality1 }
      }
    UNION
      { ?s  a              dbr:Professor ;
            dbp:birthDate  ?birthDate ;
            dbo:abstract   ?abstract
        FILTER ( ( ( ( regex(?abstract, "lawyer", "i") || regex(?abstract, "jurist", "i") ) || regex(?abstract, "juriste", "i") ) || regex(?abstract, "attorney", "i") ) || regex(?abstract, "legal professional", "i") )
        OPTIONAL
          { ?s  dbp:name  ?name
            FILTER ( xsd:string(?name) != "Marcus Tullius Cicero" )
            FILTER ( xsd:string(?name) != "Samuel Bowles" )
          }
        FILTER ( xsd:date(?Birth_Date) > "1770-01-01"^^xsd:date )
        OPTIONAL
          { ?s  owl:sameAs  ?uri
            FILTER regex(?uri, "viaf", "i")
          }
        OPTIONAL
          { ?s  dbp:birthPlace  ?bp }
        OPTIONAL
          { ?s  dbp:deathPlace  ?dp }
        OPTIONAL
          { ?s dbp:deathDate  ?deathDate }
        OPTIONAL
          { ?s  dbp:nationality  ?nationality1 }
      }
    BIND(strafter(str(?bp), "http://dbpedia.org/resource/") AS ?bp1)
    BIND(replace(str(?bp1), "[_]", " ") AS ?placeOfBirth)

    BIND(strafter(str(?dp), "http://dbpedia.org/resource/") AS ?dp1)
    BIND(replace(str(?dp1), "[_]", " ") AS ?placeOfDeath)

    BIND(strafter(str(?nationality1), "http://dbpedia.org/resource/") AS ?nationality2)
    BIND(replace(str(?nationality2), "[_]", " ") AS ?nationality)
  }
ORDER BY DESC(?uri)

"""

In this query, we have made the choice to aggregate, by a UNION clause, several queries to maximise the results' number. Also we request the "economists" and the "jurists" in only one query. 

Obviously, we chose classes and instances directly related to our population, but also the "professor" instance, because some "economists" or "jurists" are in this instance (we have tried with and without them, and there more result when we use them). 

Also, we exclude all classes because they don't add more result, except the "Economist" class (we keep it) 

For exemple, we exclude the resource "personFunction" and the resource "Jurists" because they add no more data. Additionally, we keep only the "Professor" instance for the jurists (it returns result only for the jurists).

In [56]:
sparql = SPARQLWrapper("https://dbpedia.org/sparql")  ## returnFormat=RDFXML)  [LOCALHOST]

In [57]:
sparql.setQuery(query_2)
sparql.setReturnFormat(JSON)

In [58]:
rc_db = sparql.queryAndConvert()

In [59]:
# Number of rows in the result
len(rc_db['results']['bindings'])

10000

In [60]:
# Inspect the first three rows
i = 0
for l in rc_db['results']['bindings']:
    if i <100:
        print(l)
        i += 1

{'s': {'type': 'uri', 'value': 'http://dbpedia.org/resource/António_de_Almeida_Santos'}, 'uri': {'type': 'uri', 'value': 'http://viaf.org/viaf/99921066'}, 'name': {'type': 'literal', 'xml:lang': 'en', 'value': 'António de Almeida Santos'}, 'birthDate': {'type': 'typed-literal', 'datatype': 'http://www.w3.org/2001/XMLSchema#date', 'value': '1926-02-15'}, 'deathDate': {'type': 'typed-literal', 'datatype': 'http://www.w3.org/2001/XMLSchema#date', 'value': '2016-01-18'}}
{'s': {'type': 'uri', 'value': 'http://dbpedia.org/resource/Carlos_Carvalhas'}, 'uri': {'type': 'uri', 'value': 'http://viaf.org/viaf/99826658'}, 'name': {'type': 'literal', 'xml:lang': 'en', 'value': 'Carlos Carvalhas'}, 'birthDate': {'type': 'typed-literal', 'datatype': 'http://www.w3.org/2001/XMLSchema#date', 'value': '1941-11-09'}, 'placeOfBirth': {'type': 'literal', 'value': 'São Pedro do Sul, Portugal'}}
{'s': {'type': 'uri', 'value': 'http://dbpedia.org/resource/Anita_Augspurg'}, 'uri': {'type': 'uri', 'value': 'htt

In [61]:
# Create a list with URI, VIAF URI, name, year
result_dbpedia = []
for l in rc_db['results']['bindings']:
        try: 
            name = l['name']['value']
        except Exception as e:
            name = ''
        try: 
            uri = l['uri']['value']
        except Exception as e:
            uri = ''
        try: 
            birthDate = l['birthDate']['value']
        except Exception as e:
            birthDate = ''
        try: 
            deathDate = l['deathDate']['value']
        except Exception as e:
            deathDate = ''
        try:
            placeOfBirth = l['placeOfBirth']['value']
        except Exception as e:
            placeOfBirth = ''
        try:
            placeOfDeath = l['placeOfDeath']['value']
        except Exception as e:
            placeOfDeath = ''
        try:
            nationality = l['nationality']['value']
        except Exception as e:
            nationality = ''
        result_dbpedia.append([l['s']['value'], uri, name, birthDate,deathDate, placeOfBirth, placeOfDeath])

In [62]:
# Inspect the first three of the list
result_dbpedia[:10]

[['http://dbpedia.org/resource/António_de_Almeida_Santos',
  'http://viaf.org/viaf/99921066',
  'António de Almeida Santos',
  '1926-02-15',
  '2016-01-18',
  '',
  ''],
 ['http://dbpedia.org/resource/Carlos_Carvalhas',
  'http://viaf.org/viaf/99826658',
  'Carlos Carvalhas',
  '1941-11-09',
  '',
  'São Pedro do Sul, Portugal',
  ''],
 ['http://dbpedia.org/resource/Anita_Augspurg',
  'http://viaf.org/viaf/9976800',
  'Anita Augspurg',
  '1857-09-22',
  '1943-12-20',
  '',
  ''],
 ['http://dbpedia.org/resource/Paulo_Portas',
  'http://viaf.org/viaf/99455673',
  'Paulo Portas',
  '1962-09-12',
  '',
  'Lisbon',
  ''],
 ['http://dbpedia.org/resource/Paulo_Portas',
  'http://viaf.org/viaf/99455673',
  'Paulo Portas',
  '1962-09-12',
  '',
  'Portugal',
  ''],
 ['http://dbpedia.org/resource/Pedro_Aspe',
  'http://viaf.org/viaf/9928165',
  '',
  '1950-07-07',
  '',
  'Mexico City',
  ''],
 ['http://dbpedia.org/resource/Pedro_Aspe',
  'http://viaf.org/viaf/9928165',
  '',
  '1950-07-07',
  '

# Query economists and jurists on Wikidata

In [31]:
query= """
PREFIX  bd:   <http://www.bigdata.com/rdf#>
PREFIX  wdt:  <http://www.wikidata.org/prop/direct/>
PREFIX  wikibase: <http://wikiba.se/ontology#>
PREFIX  xsd:  <http://www.w3.org/2001/XMLSchema#>
PREFIX  rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX  wd:   <http://www.wikidata.org/entity/>

SELECT DISTINCT  ?s ?name ?birthDate ?deathDate ?birthPlaceLabel ?deathPlaceLabel ?uri ?uri_bnf
WHERE
  {   { ?s  wdt:P106  wd:Q188094 } # Economists
    UNION
      { ?s  wdt:P106  wd:Q185351 } # Jurists
    OPTIONAL
      { ?s  wdt:P569  ?birthDate } 
    OPTIONAL
      { ?s  wdt:P570  ?deathDate }
    OPTIONAL
      { ?s  wdt:P19  ?birthPlace
        SERVICE wikibase:label # The use of the SERVICE clause is very important to display the property label. Also, in the SELECT, It must have "Label" used to work.
          { bd:serviceParam
                      wikibase:language  "en"
          }
      }
    OPTIONAL
      { ?s  wdt:P20  ?deathPlace
        SERVICE wikibase:label
          { bd:serviceParam
                      wikibase:language  "en"
          }
      }
    OPTIONAL
      { ?s wdt:P268 ?uri_bnf1
    BIND(uri(concat("http://data.bnf.fr/ark:/12148/cb", strafter(str(?uri_bnf1), ""))) AS ?uri_bnf2)
    BIND(concat(str(?uri_bnf2) , "#about") AS ?uri_bnf)
      }
      # We get the BnF propriety to merge easier but it returns only this URI (ex: 134841632)
      # but we need the complete URL (ex:http://data.bnf.fr/ark:/12148/cb134841632#about') to realise the merge
      # so we use the concatenation.
    OPTIONAL
      { ?s  wdt:P214  ?oldURI
        BIND(uri(concat("http://viaf.org/viaf/", strafter(str(?oldURI), ""))) AS ?uri)
      }
    FILTER ( ?birthDate >= "1770-01-01"^^xsd:dateTime )
    OPTIONAL
      { ?s  rdfs:label  ?name
        FILTER ( lang(?name) = "en" )
      }
  }
"""

In [32]:
sparql = SPARQLWrapper("https://query.wikidata.org/sparql")  ## returnFormat=RDFXML)  [LOCALHOST]

In [33]:
sparql.setQuery(query)
sparql.setReturnFormat(JSON)

In [34]:
rc_wk = sparql.queryAndConvert()

In [37]:
# Number of rows in the result
len(rc_wk['results']['bindings'])

56478

In [89]:
# Create a list with URI, VIAF URI, name, year
result_wikidata = []
for l in rc_wk['results']['bindings']:
        try: 
            name = l['name']['value']
        except Exception as e:
            name = ''
        try: 
            uri = l['uri']['value']
        except Exception as e:
            uri = ''
        try: 
            birthDate = l['birthDate']['value']
        except Exception as e:
            birthDate = ''
        try: 
            deathDate = l['deathDate']['value']
        except Exception as e:
            deathDate = ''
        try:
            birthPlace = l['birthPlaceLabel']['value']
        except Exception as e:
            birthPlace = ''
        try:
            deathPlace = l['deathPlaceLabel']['value']
        except Exception as e:
            deathPlace = ''
        try:
            uri_bnf = l['uri_bnf']['value']
        except Exception as e:
            uri_bnf = ''
        result_wikidata.append([l['s']['value'], name, birthDate,deathDate, birthPlace, deathPlace, uri, uri_bnf])

In [90]:
result_wikidata[:10]

[['http://www.wikidata.org/entity/Q77404',
  'Ingeborg Schwenzer',
  '1951-10-25T00:00:00Z',
  '',
  'Stuttgart',
  '',
  'http://viaf.org/viaf/91748910',
  'http://data.bnf.fr/ark:/12148/cb15067859n#about'],
 ['http://www.wikidata.org/entity/Q77341',
  'Hans Globke',
  '1898-09-10T00:00:00Z',
  '1973-02-13T00:00:00Z',
  'Düsseldorf',
  'Bad Godesberg',
  'http://viaf.org/viaf/54939901',
  'http://data.bnf.fr/ark:/12148/cb15597313f#about'],
 ['http://www.wikidata.org/entity/Q77390',
  'Christoph Ahlhaus',
  '1969-08-28T00:00:00Z',
  '',
  'Heidelberg',
  '',
  'http://viaf.org/viaf/171463495',
  ''],
 ['http://www.wikidata.org/entity/Q89299',
  'Emil Schlagintweit',
  '1835-07-07T00:00:00Z',
  '1904-10-29T00:00:00Z',
  'Munich',
  'Zweibrücken',
  'http://viaf.org/viaf/74008741',
  'http://data.bnf.fr/ark:/12148/cb13519131c#about'],
 ['http://www.wikidata.org/entity/Q72628',
  'Alfred von Kiderlen-Waechter',
  '1852-07-10T00:00:00Z',
  '1912-12-30T00:00:00Z',
  'Stuttgart',
  'Stuttgar

# Dataframes

The script below serves to change lists into a dataframe.

In [63]:
df_bnf = pd.DataFrame(result_bnf, columns=['uri_bnf', 'viaf', 'name_bnf', 'Sname','dateBirth_bnf','dateDeath_bnf' , 'placeOfBirth_bnf','placeOfDeath_bnf','bio_bnf'])
print(len(df_bnf))
df_bnf.fillna('')

df_bnf[:10]

11132


,uri_bnf,viaf,name_bnf,Sname,dateBirth_bnf,dateDeath_bnf,placeOfBirth_bnf,placeOfDeath_bnf,bio_bnf
0,http://data.bnf.fr/ark:/12148/cb12981404c#about,http://viaf.org/viaf/99996033,Léon Garnier,,1836-11-10,1901-05-06,,,Juriste. - Administrateur et homme de lettres....
1,http://data.bnf.fr/ark:/12148/cb13484444m#about,http://viaf.org/viaf/9999219,Gaston de Pawlowski,,1874-06-14,1933-02-02,Joigny (Yonne),Paris,Docteur en droit. - Critique littéraire et thé...
2,http://data.bnf.fr/ark:/12148/cb134841632#about,http://viaf.org/viaf/9999131,Jean-Michel Berton,,1794-07-03,1845-10-20,Cahors (Lot),,"Écrivain politique, avocat à la Cour de cassat..."
3,http://data.bnf.fr/ark:/12148/cb13379520q#about,http://viaf.org/viaf/9995247,Emmanuel Mathieu,,1852-07-19,19..,,,"Docteur en droit (Paris, 1873)"
4,http://data.bnf.fr/ark:/12148/cb13338312g#about,http://viaf.org/viaf/9994322,Josiah Henry Benton,,1843,1917,,,Juriste. - Bibliophile
5,http://data.bnf.fr/ark:/12148/cb13322315v#about,http://viaf.org/viaf/9991357,Paul Pic,,1862-05-31,1944-08-16,Alger,"Hartheim, Autriche",Juriste. - Professeur de droit à la Faculté de...
6,http://data.bnf.fr/ark:/12148/cb13193319k#about,http://viaf.org/viaf/9989230,Gaston Ravisse,,1877-05-15,1935-06-25,Calais (Pas-de-Calais),Paris (France),Avocat. - Spécialiste du monde de l'entreprise...
7,http://data.bnf.fr/ark:/12148/cb15042710d#about,http://viaf.org/viaf/99857689,Cândido Jucá Filho,,1900,1982,,,Avocat
8,http://data.bnf.fr/ark:/12148/cb13169620f#about,http://viaf.org/viaf/9985289,Joseph de Trémaudan,,1846,19..,,,Juge à Paimboeuf. - Historien local
9,http://data.bnf.fr/ark:/12148/cb13075767f#about,http://viaf.org/viaf/9982622,Achille Villey-Desmeserets,,1878-11-06,1955-12-08,Caen (Calvados),Paris (France),Avocat. - Préfet


In [64]:
# Drop duplicates lines and keep first
df_bnf.drop_duplicates(subset ="uri_bnf", keep = 'first', inplace=True)
print(len(df_bnf))

11039


In [92]:
# Create an id to Bnf Data dataframe
df_bnf["id_bnf"] = df_bnf.index + 0
df_bnf= pd.DataFrame(df_bnf, columns=['uri_bnf', 'viaf', 'name_bnf', 'Sname','dateBirth_bnf','dateDeath_bnf' , 'placeOfBirth_bnf','placeOfDeath_bnf','bio_bnf'],index=df_bnf["id_bnf"])
df_bnf[-20:]

NameError: name 'df_bnf' is not defined

In [66]:
df_dbp = pd.DataFrame(result_dbpedia, columns=['uri_dbp', 'viaf', 'name_dbp','birthDate_dbp','deathDate_dbp' , 'placeOfBirth_dbp','placeOfDeath_dbp'])
print(len(df_dbp))
df_dbp.fillna('')
df_dbp.head(20)

10000


,uri_dbp,viaf,name_dbp,birthDate_dbp,deathDate_dbp,placeOfBirth_dbp,placeOfDeath_dbp
0,http://dbpedia.org/resource/António_de_Almeida...,http://viaf.org/viaf/99921066,António de Almeida Santos,1926-02-15,2016-01-18,,
1,http://dbpedia.org/resource/Carlos_Carvalhas,http://viaf.org/viaf/99826658,Carlos Carvalhas,1941-11-09,,"São Pedro do Sul, Portugal",
2,http://dbpedia.org/resource/Anita_Augspurg,http://viaf.org/viaf/9976800,Anita Augspurg,1857-09-22,1943-12-20,,
3,http://dbpedia.org/resource/Paulo_Portas,http://viaf.org/viaf/99455673,Paulo Portas,1962-09-12,,Lisbon,
4,http://dbpedia.org/resource/Paulo_Portas,http://viaf.org/viaf/99455673,Paulo Portas,1962-09-12,,Portugal,
5,http://dbpedia.org/resource/Pedro_Aspe,http://viaf.org/viaf/9928165,,1950-07-07,,Mexico City,
6,http://dbpedia.org/resource/Pedro_Aspe,http://viaf.org/viaf/9928165,,1950-07-07,,Mexico,
7,http://dbpedia.org/resource/Fernando_Teixeira_...,http://viaf.org/viaf/99275725,Fernando Teixeira dos Santos,1951-09-13,,"Maia, Portugal",
8,http://dbpedia.org/resource/Fernando_Teixeira_...,http://viaf.org/viaf/99275725,Fernando Teixeira dos Santos,1951-09-13,,Portugal,
9,http://dbpedia.org/resource/Xavier_Vives,http://viaf.org/viaf/9920331,Xavier Vives,1955-01-23,,,


In [67]:
# Drop duplicates lines and keep first
df_dbp.drop_duplicates(subset =["uri_dbp"], keep = 'first', inplace=True)
print(len(df_dbp))
df_dbp.head()

4383


,uri_dbp,viaf,name_dbp,birthDate_dbp,deathDate_dbp,placeOfBirth_dbp,placeOfDeath_dbp
0,http://dbpedia.org/resource/António_de_Almeida...,http://viaf.org/viaf/99921066,António de Almeida Santos,1926-02-15,2016-01-18,,
1,http://dbpedia.org/resource/Carlos_Carvalhas,http://viaf.org/viaf/99826658,Carlos Carvalhas,1941-11-09,,"São Pedro do Sul, Portugal",
2,http://dbpedia.org/resource/Anita_Augspurg,http://viaf.org/viaf/9976800,Anita Augspurg,1857-09-22,1943-12-20,,
3,http://dbpedia.org/resource/Paulo_Portas,http://viaf.org/viaf/99455673,Paulo Portas,1962-09-12,,Lisbon,
5,http://dbpedia.org/resource/Pedro_Aspe,http://viaf.org/viaf/9928165,,1950-07-07,,Mexico City,


In [68]:
# Create an id to DBpedia dataframe
df_dbp["id_dbp"] = df_dbp.index + 0
df_dbp= pd.DataFrame(df_dbp, columns=['uri_dbp', 'viaf', 'name_dbp','birthDate_dbp','deathDate_dbp' , 'placeOfBirth_dbp','placeOfDeath_dbp'],index=df_dbp["id_dbp"])
print(len(df_dbp))
df_dbp[-20:]

4383


,uri_dbp,viaf,name_dbp,birthDate_dbp,deathDate_dbp,placeOfBirth_dbp,placeOfDeath_dbp
id_dbp,,,,,,,
9968,http://dbpedia.org/resource/David_Kaushansky,,David Moiseevich Kaushansky,1893-03-15,after 1950,Chișinău,
9969,http://dbpedia.org/resource/Benjamin_Cudworth_...,,Benjamin Cudworth Yancey Jr,1817-04-27,--10-24,United States,
9971,http://dbpedia.org/resource/Leo_Koretz,,Leo Koretz,1879-07-30,1925-01-08,Bohemia,
9973,http://dbpedia.org/resource/Leopoldo_Bravo,,Leopoldo Bravo,1919-03-15,2006-08-04,"San Juan, Argentina",
9974,http://dbpedia.org/resource/Terry_Nugent,,"Terence James ""Terry"" Nugent",1920-12-09,2006-04-13,,
9975,http://dbpedia.org/resource/Henry_C._Slemp,,Henry C. Slemp,1881-03-26,1901-01-11,"Seminary, Virginia",
9977,http://dbpedia.org/resource/Mitchell_Cary_Alford,,Mitchell Cary Alford,1855-07-10,1914-12-09,"Fayette County, Kentucky",
9978,http://dbpedia.org/resource/Terry_Haskins,,,1955-01-31,2000-10-24,"Oakland County, Michigan",
9981,http://dbpedia.org/resource/Bernard_M._L._Ernst,,Bernard M. L. Ernst,1879-03-17,--11-28,,


In [91]:
df_wk= pd.DataFrame(result_wikidata, columns=['uri_wk', 'name_wk', 'dateBirth_wk', "dateDeath_wk", "placeOfBirth_wk", "placeOfDeath_wk",'viaf', "uri_bnf"])
print(len(df_wk))
df_wk.fillna('')
df_wk.head()

56478


,uri_wk,name_wk,dateBirth_wk,dateDeath_wk,placeOfBirth_wk,placeOfDeath_wk,viaf,uri_bnf
0,http://www.wikidata.org/entity/Q77404,Ingeborg Schwenzer,1951-10-25T00:00:00Z,,Stuttgart,,http://viaf.org/viaf/91748910,http://data.bnf.fr/ark:/12148/cb15067859n#about
1,http://www.wikidata.org/entity/Q77341,Hans Globke,1898-09-10T00:00:00Z,1973-02-13T00:00:00Z,Düsseldorf,Bad Godesberg,http://viaf.org/viaf/54939901,http://data.bnf.fr/ark:/12148/cb15597313f#about
2,http://www.wikidata.org/entity/Q77390,Christoph Ahlhaus,1969-08-28T00:00:00Z,,Heidelberg,,http://viaf.org/viaf/171463495,
3,http://www.wikidata.org/entity/Q89299,Emil Schlagintweit,1835-07-07T00:00:00Z,1904-10-29T00:00:00Z,Munich,Zweibrücken,http://viaf.org/viaf/74008741,http://data.bnf.fr/ark:/12148/cb13519131c#about
4,http://www.wikidata.org/entity/Q72628,Alfred von Kiderlen-Waechter,1852-07-10T00:00:00Z,1912-12-30T00:00:00Z,Stuttgart,Stuttgart,http://viaf.org/viaf/54958174,http://data.bnf.fr/ark:/12148/cb135097503#about


In [92]:
# We drop 'T00:00:00Z' to have the format 'YYYY-MM-DD' to improve the merge below.
df_wk['dateBirth_wk'] = df_wk['dateBirth_wk'].str.rstrip('T00:00:00Z')
df_wk['dateDeath_wk'] = df_wk['dateDeath_wk'].str.rstrip('T00:00:00Z')


In [93]:
# Drop duplicates lines and keep first
df_wk.drop_duplicates(subset ='uri_wk', keep = 'first', inplace=True)
df_wk.fillna('')
print(len(df_wk))
df_wk.head()

53316


,uri_wk,name_wk,dateBirth_wk,dateDeath_wk,placeOfBirth_wk,placeOfDeath_wk,viaf,uri_bnf
0,http://www.wikidata.org/entity/Q77404,Ingeborg Schwenzer,1951-10-25,,Stuttgart,,http://viaf.org/viaf/91748910,http://data.bnf.fr/ark:/12148/cb15067859n#about
1,http://www.wikidata.org/entity/Q77341,Hans Globke,1898-09-1,1973-02-13,Düsseldorf,Bad Godesberg,http://viaf.org/viaf/54939901,http://data.bnf.fr/ark:/12148/cb15597313f#about
2,http://www.wikidata.org/entity/Q77390,Christoph Ahlhaus,1969-08-28,,Heidelberg,,http://viaf.org/viaf/171463495,
3,http://www.wikidata.org/entity/Q89299,Emil Schlagintweit,1835-07-07,1904-10-29,Munich,Zweibrücken,http://viaf.org/viaf/74008741,http://data.bnf.fr/ark:/12148/cb13519131c#about
4,http://www.wikidata.org/entity/Q72628,Alfred von Kiderlen-Waechter,1852-07-1,1912-12-3,Stuttgart,Stuttgart,http://viaf.org/viaf/54958174,http://data.bnf.fr/ark:/12148/cb135097503#about


In [95]:
# Create an id to Wikidata dataframe
df_wk["id_wk"] = df_wk.index + 0
df_wk= pd.DataFrame(df_wk, columns=['uri_wk', 'name_wk', 'dateBirth_wk', "dateDeath_wk", "placeOfBirth_wk", "placeOfDeath_wk",'viaf', "uri_bnf"],index=df_wk["id_wk"])
df_wk[:15]

,uri_wk,name_wk,dateBirth_wk,dateDeath_wk,placeOfBirth_wk,placeOfDeath_wk,viaf,uri_bnf
id_wk,,,,,,,,
0,http://www.wikidata.org/entity/Q77404,Ingeborg Schwenzer,1951-10-25,,Stuttgart,,http://viaf.org/viaf/91748910,http://data.bnf.fr/ark:/12148/cb15067859n#about
1,http://www.wikidata.org/entity/Q77341,Hans Globke,1898-09-1,1973-02-13,Düsseldorf,Bad Godesberg,http://viaf.org/viaf/54939901,http://data.bnf.fr/ark:/12148/cb15597313f#about
2,http://www.wikidata.org/entity/Q77390,Christoph Ahlhaus,1969-08-28,,Heidelberg,,http://viaf.org/viaf/171463495,
3,http://www.wikidata.org/entity/Q89299,Emil Schlagintweit,1835-07-07,1904-10-29,Munich,Zweibrücken,http://viaf.org/viaf/74008741,http://data.bnf.fr/ark:/12148/cb13519131c#about
4,http://www.wikidata.org/entity/Q72628,Alfred von Kiderlen-Waechter,1852-07-1,1912-12-3,Stuttgart,Stuttgart,http://viaf.org/viaf/54958174,http://data.bnf.fr/ark:/12148/cb135097503#about
5,http://www.wikidata.org/entity/Q65539,Peter Altmaier,1958-06-18,,Ensdorf,,http://viaf.org/viaf/4137633,
6,http://www.wikidata.org/entity/Q89439,Adolf Damaschke,1865-11-24,1935-07-3,Berlin,Berlin,http://viaf.org/viaf/54949744,http://data.bnf.fr/ark:/12148/cb130056891#about
7,http://www.wikidata.org/entity/Q72654,Magnus von Braun,1878-02-07,1972-08-29,Bagrationovsk,Oberaudorf,http://viaf.org/viaf/262459734,
10,http://www.wikidata.org/entity/Q65561,Hans Apel,1932-02-25,2011-09-06,Hamburg,Hamburg,http://viaf.org/viaf/232142151,http://data.bnf.fr/ark:/12148/cb121740063#about


deathDate_dbp# RecordLinkage

In [96]:
from pathlib import Path
import fuzzymatcher

In [77]:
# We use the dataframes in csv to don't run again requests
df_bnf.to_csv("df_bnf.csv")
df_wk.to_csv("df_wk.csv")
df_dbp.to_csv("df_dbp.csv")

NameError: name 'df_bnf' is not defined

In [571]:
BnF_Data = pd.read_csv('df_bnf.csv')
Wikidata = pd.read_csv('df_wk.csv')
DBpedia = pd.read_csv('df_dbp.csv')

# Merge

## 1- Between Wikidata and BnF Data

In [572]:
# We realize a merge between Wikidata and BnF Data using the BnF URIs in common
merged_bnf_wk = pd.merge( BnF_Data, Wikidata , on='uri_bnf', how='inner', sort='uri_bnf')
print("")
print("This method merge", len(merged_bnf_wk), "persons.")
print("")
cols=["id_bnf","id_wk","uri_bnf","uri_wk","name_bnf","dateBirth_bnf","dateDeath_bnf","placeOfBirth_bnf", "placeOfDeath_bnf", "name_wk", "dateBirth_wk","dateDeath_wk","placeOfBirth_wk","placeOfDeath_wk"]
merged_bnf_wk[cols][:10]


This method merge 1558 persons.



,id_bnf,id_wk,uri_bnf,uri_wk,name_bnf,dateBirth_bnf,dateDeath_bnf,placeOfBirth_bnf,placeOfDeath_bnf,name_wk,dateBirth_wk,dateDeath_wk,placeOfBirth_wk,placeOfDeath_wk
0,1713,6505,http://data.bnf.fr/ark:/12148/cb10071436z#about,http://www.wikidata.org/entity/Q1360518,Ernst Walz,1859-07-18,1941-12-18,"Heidelberg (Bade-Wurtemberg, Allemagne)","Heidelberg (Bade-Wurtemberg, Allemagne)",Ernst Walz,1859-07-19,1941-12-18,Heidelberg,Heidelberg
1,6594,28290,http://data.bnf.fr/ark:/12148/cb101827728#about,http://www.wikidata.org/entity/Q15177628,Adrien Calmètes,1800-09-19,1871-02-27,Figueras (Espagne),Montpellier (Hérault),Adrien Calmètes,1800-09-19,1871-02-27,Figueres,Montpellier
2,6600,44446,http://data.bnf.fr/ark:/12148/cb10190862z#about,http://www.wikidata.org/entity/Q56646471,Béla Kun,1861-04-24,1934-09-19,"Sátoraljaújhelyi, Hongrie","Budapest, Hongrie",Kun Béla,1861-04-24,1934-09-19,Sátoraljaújhely,Budapest District VII
3,6929,6082,http://data.bnf.fr/ark:/12148/cb102042826#about,http://www.wikidata.org/entity/Q1214240,Imre Nagy,1822-07-01,1894-05-05,"Németkeresztúr, aujourd'hui Deutschkreutz, Aut...",Budapest,Imre Nagy,1822-06-01,1894-05-05,Deutschkreutz,Budapest
4,6931,46637,http://data.bnf.fr/ark:/12148/cb10207440j#about,http://www.wikidata.org/entity/Q94851244,Wilhelm Gustav Karl Starke,1824-02-26,1903-03-10,"Lubán (Prusse, aujourd'hui Pologne)",Berlin (Allemagne),Wilhelm Gustav Karl Starke,1824-02-26,1903-03-09,Lubań,Berlin
5,7694,2149,http://data.bnf.fr/ark:/12148/cb102101229#about,http://www.wikidata.org/entity/Q343248,Karel Baxa,1863-06-23,1938-01-05,NaN,NaN,Karel Baxa,1862-06-24,1938-01-05,Sedlčany,Prague
6,3976,28075,http://data.bnf.fr/ark:/12148/cb10217219c#about,http://www.wikidata.org/entity/Q15830136,Jan Heller,1848-11-13,1932-03-20,Vranov u Rokycan (République tchèque),Prague (République tchèque),Jan Heller,1848-11-13,1932-03-2,NaN,Prague
7,1011,15408,http://data.bnf.fr/ark:/12148/cb102252805#about,http://www.wikidata.org/entity/Q3840074,Luigi Rava,1860-12-01,1938-05-12,"Ravenne, Italie",Rome,Luigi Rava,1860-11-29,1938-05-12,Ravenna,Rome
8,7407,45126,http://data.bnf.fr/ark:/12148/cb10226211k#about,http://www.wikidata.org/entity/Q57202321,André Roux,1893-10-10,19..,Privas (Ardèche),NaN,André Louis Roux,1893-10-1,NaN,Privas,NaN
9,10383,29226,http://data.bnf.fr/ark:/12148/cb10226945t#about,http://www.wikidata.org/entity/Q13411514,Arveds Švābe,1888-05-25,1959-08-20,NaN,NaN,Arveds Švābe,1888-05-25,1959-08-2,Q16362410,Stockholm


## first method of recordlinkage: fuzzymatcher

It permits to calculate a match score between to strings. Here, we use the "fuzzymatcher" library.

This article explains  very well how uses it. cf. https://pbpython.com/record-linking.html

There is also a documentation but it is very light. cf. https://fuzzymatcher.readthedocs.io/en/latest/

It seems work well only for the strings, because I guess it manages integers as strings. 

In [536]:
left_on=["name_bnf", "dateBirth_bnf","dateDeath_bnf"]
right_on=["name_dbp", "birthDate_dbp","deathDate_dbp"]

In [ ]:
matched_results = fuzzymatcher.fuzzy_left_join(BnF_Data,
                                            DBpedia,
                                            left_on,
                                            right_on,
                                            left_id_col='uri_bnf',
                                            right_id_col='uri_dbp')

In [106]:
cols_bnf_dbp= ["best_match_score","id_bnf","uri_bnf","viaf_bnf", "name_bnf", "dateBirth_bnf", "dateDeath_bnf","placeOfBirth_bnf","placeOfDeath_bnf", "bio_bnf", "id_dbp","uri_dbp", "viaf_dbp", "name_dbp","birthDate_dbp","deathDate_dbp", "placeOfBirth_dbp","placeOfDeath_dbp"]

In [107]:
best_match_bnf_dbp=matched_results[cols_bnf_dbp].sort_values(by=["best_match_score"], ascending=False).head(10)
best_match_bnf_dbp

,best_match_score,id_bnf,uri_bnf,viaf_bnf,name_bnf,dateBirth_bnf,dateDeath_bnf,placeOfBirth_bnf,placeOfDeath_bnf,bio_bnf,id_dbp,uri_dbp,viaf_dbp,name_dbp,birthDate_dbp,deathDate_dbp,placeOfBirth_dbp,placeOfDeath_dbp
249451,1.752024,4716,http://data.bnf.fr/ark:/12148/cb11298933w#about,http://viaf.org/viaf/31990071,Auguste Champetier de Ribes,1882,1947.0,Antony (Hauts-de-Seine),Paris,Homme politique. - Avocat près la cour d'appel...,2521.0,http://dbpedia.org/resource/Auguste_Champetier...,http://viaf.org/viaf/31990071,Auguste Champetier de Ribes,1882-07-30,1947-03-06,"Antony, Hauts-de-Seine",NaN
545878,1.213143,9991,http://data.bnf.fr/ark:/12148/cb12649301n#about,NaN,Anísio Teixeira,1900,1971.0,"Caetité, Brésil","Rio de Janeiro, Brésil",Juriste. - Éducateur,5875.0,http://dbpedia.org/resource/Anísio_Teixeira,NaN,Anísio Teixeira,1900-07-11,1971-03-11,Brazil,Rio de Janeiro
66943,1.201671,1257,http://data.bnf.fr/ark:/12148/cb11926733m#about,http://viaf.org/viaf/73860740,Jean-Louis Tixier-Vignancour,1907,1989.0,NaN,NaN,Avocat. - Ancien député. - Candidat à l'électi...,8325.0,http://dbpedia.org/resource/Jean-Louis_Tixier-...,NaN,Jean-Louis Tixier-Vignancour,1907-10-12,1989-09-29,NaN,NaN
522148,1.043139,9539,http://data.bnf.fr/ark:/12148/cb108125478#about,NaN,Luigi Amoroso,1886,1965.0,Naples,Rome,"Mathématicien, professeur d'économie politique...",35.0,http://dbpedia.org/resource/Luigi_Amoroso,http://viaf.org/viaf/9838425,Luigi Amoroso,1886-03-26,1965-10-28,Naples,Rome
94788,0.978183,1779,http://data.bnf.fr/ark:/12148/cb12599024b#about,http://viaf.org/viaf/66585190,Michel Crépeau,1930,1999.0,Fontenay-le-Comte (Vendée),Paris,Avocat. - Homme politique. - Co-fondateur puis...,9544.0,http://dbpedia.org/resource/Michel_Crépeau,NaN,Michel Crépeau,1930-10-30,1999-03-30,Fontenay-le-Comte,NaN
227365,0.950051,4283,http://data.bnf.fr/ark:/12148/cb120792656#about,http://viaf.org/viaf/36942970,Gustav Radbruch,1878,1949.0,Lübeck (Allemagne),Heidelberg (Allemagne),Homme politique. - Philosophe du droit. - Juriste,2340.0,http://dbpedia.org/resource/Gustav_Radbruch,http://viaf.org/viaf/36942970,Gustav Radbruch,1878-11-21,1949-11-23,Lübeck,Heidelberg
137886,0.882604,2607,http://data.bnf.fr/ark:/12148/cb123419831#about,http://viaf.org/viaf/56683086,William Martin Geldart,1870,1922.0,NaN,NaN,"Membre de Trinity College, Oxford, GB. - Juriste",1493.0,http://dbpedia.org/resource/William_Martin_Gel...,http://viaf.org/viaf/56683086,William Martin Geldart,1870-06-07,--02-12,NaN,NaN
520575,0.869083,9499,http://data.bnf.fr/ark:/12148/cb12278698c#about,NaN,Gustav Cassel,1866,1945.0,Stockholm,Joenkoping (Suède),"Docteur ès sciences de l'Université d'Uppsala,...",1808.0,http://dbpedia.org/resource/Gustav_Cassel,http://viaf.org/viaf/49285903,Gustav Cassel,1866-10-20,1945-01-14,Stockholm,Jönköping
366919,0.825417,6738,http://data.bnf.fr/ark:/12148/cb12343132d#about,http://viaf.org/viaf/22214944,Benjamin Nathan Cardozo,1870,1938.0,NaN,NaN,Juriste. - A été attaché à la Cour Suprême des...,3372.0,http://dbpedia.org/resource/Benjamin_N._Cardozo,http://viaf.org/viaf/22214944,Benjamin Nathan Cardozo,1870-05-24,1938-07-09,NaN,NaN
186169,0.818944,3539,http://data.bnf.fr/ark:/12148/cb11047680z#about,http://viaf.org/viaf/46753389,Camille Blaisot,1881,1945.0,Valognes (Manche),Dachau (Allemagne),Avocat à la Cour d'appel de Caen. - A été dépu...,8182.0,http://dbpedia.org/resource/Camille_Blaisot,NaN,Camille Blaisot,1881-01-19,1945-01-24,Valognes,"Dachau, Bavaria"


In [108]:
maReSo=matched_results[cols_bnf_dbp].sort_values(by=["best_match_score"], ascending=True).head(10)
maReSo

,best_match_score,id_bnf,uri_bnf,viaf_bnf,name_bnf,dateBirth_bnf,dateDeath_bnf,placeOfBirth_bnf,placeOfDeath_bnf,bio_bnf,id_dbp,uri_dbp,viaf_dbp,name_dbp,birthDate_dbp,deathDate_dbp,placeOfBirth_dbp,placeOfDeath_dbp
160996,-2.591975,3062,http://data.bnf.fr/ark:/12148/cb136243015#about,http://viaf.org/viaf/5109006,Menaḥem Zevi Kaddari,1925,2011.0,Mezőkövesd (Hongrie),Ramat-Gan (Israël),"Linguiste. - Professeur émérite, Department of...",1908.0,http://dbpedia.org/resource/Carl_Ernst_Fürst_F...,http://viaf.org/viaf/47524071,Carl Ernst Fürst Fugger von Glött,1859-07-02,1940-04-25,Oberndorf am Lech,Kirchheim in Schwaben
605865,-2.460501,10982,http://data.bnf.fr/ark:/12148/cb17791066c#about,NaN,Sayf ibn Ḥamad ibn Šayẖān ibn Muḥammad ibn ...,1892,1961.0,"Sima, Azki - Oman","Matrah, Mascate - Oman",Poète. - Juge,6057.0,http://dbpedia.org/resource/Ahmed_Ali_Sheikh,NaN,Ahmed Ali M Sheikh,1961-10-03,NaN,NaN,NaN
189849,-2.333900,3606,http://data.bnf.fr/ark:/12148/cb16322024d#about,http://viaf.org/viaf/44752143,Maximilian Brantl,1881,1951.0,Munich (Allemagne),Prien am Chiemsee (Allemagne),"Écrivain, poète et parolier. - A aussi utilisé...",1908.0,http://dbpedia.org/resource/Carl_Ernst_Fürst_F...,http://viaf.org/viaf/47524071,Carl Ernst Fürst Fugger von Glött,1859-07-02,1940-04-25,Oberndorf am Lech,Kirchheim in Schwaben
41512,-2.079271,790,http://data.bnf.fr/ark:/12148/cb121974159#about,http://viaf.org/viaf/78772873,Franc Miklošič,1813,1891.0,"Radomerščak dans Ljutomer, Styrie (Empire ausr...","Vienne (Empire ausrto-hongrois, aujourd'hui Au...",Docteur en philosophie en 1838 (de l'Universit...,3919.0,http://dbpedia.org/resource/Julius_von_Kirchmann,http://viaf.org/viaf/13104809,NaN,1802-11-05,1884-10-20,NaN,German Empire
198924,-2.042366,3763,http://data.bnf.fr/ark:/12148/cb15984663k#about,http://viaf.org/viaf/4322733,Sándor Boschan,1891,1942.0,"Ada, Comitat Bacs-Bodrog, Hongrie","Csurog, [avant 1920 dans le comitat de Bacs-Bo...",Écrivain. - Avocat à Csurog. - Tué lors de la ...,1547.0,http://dbpedia.org/resource/Sándor_Wekerle,http://viaf.org/viaf/54951146,Sándor Wekerle,1848-11-14,1921-08-26,Kingdom of Hungary,Kingdom of Hungary (1920–1946)
434551,-1.837949,7890,http://data.bnf.fr/ark:/12148/cb11229242q#about,http://viaf.org/viaf/160351646,Milivoje Perović,1912,1975.0,Mala Braina près de Medveđa (Royaume de Serbie...,"Belgrade (Yougoslavie, aujourd'hui Serbie)",Docteur en droit. - Écrivain,2169.0,http://dbpedia.org/resource/Smilja_Avramov,http://viaf.org/viaf/41910880,Smilja Avramov,1918-02-15,2018-10-02,Austria-Hungary,Belgrade
602073,-1.785585,10919,http://data.bnf.fr/ark:/12148/cb17708334d#about,NaN,Nicasio Idar,1955,1914.0,"Point Isabel (Tx., États-Unis)","Laredo (Tx., État-Unis)",Journaliste et avocat. Défenseur des droits ci...,7846.0,http://dbpedia.org/resource/Charles_Wingender,NaN,http://dbpedia.org/resource/Denver_Pioneers_fo...,1884-09-20,1943,"Mineral Point, Wisconsin",NaN
32323,-1.782468,596,http://data.bnf.fr/ark:/12148/cb16184017h#about,http://viaf.org/viaf/84288151,Mehmed Spaho,1883,1939.0,"Sarajevo (Empire austro-hongrois, aujourd'hui ...","Sarajevo (Royaume de Yougoslavie, aujourd'hui ...",Docteur en droit. - Homme politique et avocat....,7885.0,http://dbpedia.org/resource/Alen_Hujić,NaN,Alen Hujić,1967-05-11,NaN,Sarajevo,NaN
273005,-1.767561,5109,http://data.bnf.fr/ark:/12148/cb10818445r#about,http://viaf.org/viaf/310510743,Etbin Henrik Costa,1832,1875.0,"Novo mesto (Empire austro-hongrois, aujourd'hu...","Ljubljana (Empire austro-hongrois, aujourd'hui...","Bibliographe, avocat et homme politique",5031.0,http://dbpedia.org/resource/Rui_Costa_(politic...,NaN,Rui Costa,1963-01-18,NaN,NaN,NaN
296239,-1.764684,5509,http://data.bnf.fr/ark:/12148/cb12025353g#about,http://viaf.org/viaf/29547542,Neoklī́s Kazázīs,1849,1936.0,"Pétra Lésvou (Mytilène/Lesbos, Îles Égéennes s...","Athènes (Attique, Grèce)",Juriste. - Professeur d'économie politique (18...,1540.0,http://dbpedia.org/resource/Henry_Farber,http://viaf.org/viaf/5

In [115]:
matched_bnf_dbp=matched_results[cols_bnf_dbp].query("best_match_score <= .05").sort_values(
    by=['best_match_score'], ascending=False)
print(len(matched_bnf_dbp))
matched_bnf_dbp[:10]

10336


,best_match_score,id_bnf,uri_bnf,viaf_bnf,name_bnf,dateBirth_bnf,dateDeath_bnf,placeOfBirth_bnf,placeOfDeath_bnf,bio_bnf,id_dbp,uri_dbp,viaf_dbp,name_dbp,birthDate_dbp,deathDate_dbp,placeOfBirth_dbp,placeOfDeath_dbp
261401,0.049958,4914,http://data.bnf.fr/ark:/12148/cb13543089k#about,http://viaf.org/viaf/316439034,José Cafranga,1780,1854.0,NaN,NaN,Juriste et homme politique espagnol,6240.0,http://dbpedia.org/resource/José_Fragelli,NaN,José Fragelli,1915-12-31,2010-04-30,NaN,NaN
564947,0.049958,10292,http://data.bnf.fr/ark:/12148/cb12958606p#about,NaN,José Castillejo,1877,1945.0,NaN,NaN,Philosophe et juriste,6240.0,http://dbpedia.org/resource/José_Fragelli,NaN,José Fragelli,1915-12-31,2010-04-30,NaN,NaN
359415,0.049958,6605,http://data.bnf.fr/ark:/12148/cb13328185x#about,http://viaf.org/viaf/226077615,José Balarello,1926,2015.0,NaN,NaN,Avocat et homme politique. - Sénateur des Alpe...,6240.0,http://dbpedia.org/resource/José_Fragelli,NaN,José Fragelli,1915-12-31,2010-04-30,NaN,NaN
188418,0.049610,3582,http://data.bnf.fr/ark:/12148/cb15918325q#about,http://viaf.org/viaf/45402214,Carl Meyer,1873,1947.0,NaN,NaN,Juriste et homme politique du canton d'Appenze...,153.0,http://dbpedia.org/resource/Laurence_Meyer,http://viaf.org/viaf/91708741,Laurence Meyer,1944-03-08,NaN,NaN,NaN
109760,0.049027,2080,http://data.bnf.fr/ark:/12148/cb10551494k#about,http://viaf.org/viaf/63874811,John Frederick Lewis,1860,1932.0,NaN,NaN,Avocat né à Philadelphie. A légué sa collectio...,5763.0,http://dbpedia.org/resource/John_Feerick,NaN,John D. Feerick,1936-07-12,NaN,NaN,NaN
535731,0.048658,9827,http://data.bnf.fr/ark:/12148/cb165642447#about,NaN,David Anisi,1949,2008.0,NaN,NaN,Economiste,9696.0,http://dbpedia.org/resource/David_Jones_(Cardi...,NaN,David Jones,1883-10-29,1966-07-28,"Johnstown, Pennsylvania",NaN
453765,0.048011,8237,http://data.bnf.fr/ark:/12148/cb12392644k#about,http://viaf.org/viaf/126145857808723020035,William Winthrop,1831,1899.0,NaN,NaN,Colonnel. - Juge militaire dans l'armée améric...,4443.0,http://dbpedia.org/resource/William_Davis_(jou...,http://viaf.org/viaf/105869848,William Davis,1933-03-06,2019-02-02,NaN,NaN
158653,0.047238,3023,http://data.bnf.fr/ark:/12148/cb10938104x#about,http://viaf.org/viaf/51681643,Georges Boyer,1896,1960.0,NaN,NaN,Juriste,7243.0,http://dbpedia.org/resource/Blair_Boyer,NaN,Blair Boyer,1981-03-30,NaN,NaN,NaN
31245,0.046638,574,http://data.bnf.fr/ark:/12148/cb10725608d#about,http://viaf.org/viaf/85931863,Joseph-Georges Demangeat,1787,1866.0,NaN,NaN,"Procureur du roi à Nantes, puis avocat général...",1768.0,http://dbpedia.org/resource/Joseph_George_Rose...,http://viaf.org/viaf/49981696,Joseph George Rosengarten,1835-07-14,1921-01-14,NaN,NaN
499983,0.046638,9074,http://data.bnf.fr/ark:/12148/cb12413087z#about,http://viaf.org/viaf/100292131,Joseph Chitty,1776,1841.0,NaN,NaN,Juriste,4290.0,http://dbpedia.org/resource/Joseph_Luns,http://viaf.org/viaf/109095225,Joseph Luns,1911-08-28,2002-07-17,NaN,NaN


In [184]:
bnf_dbp=pd.DataFrame(matched_bnf_dbp, columns=['uri_bnf','uri_dbp','viaf_bnf','name_bnf', 'year_bnf','bio_bnf'])


,uri_bnf,uri_dbp,viaf_bnf,name_bnf,year_bnf,bio_bnf
62919,http://data.bnf.fr/ark:/12148/cb121108528#about,http://dbpedia.org/resource/Jacques_Mairesse_(...,http://viaf.org/viaf/7421350,Jacques Mercier,1921,Avocat et homme politique. - Avocat à la cour ...
429849,http://data.bnf.fr/ark:/12148/cb14316906f#about,http://dbpedia.org/resource/Béla_Szászy,http://viaf.org/viaf/166146883,Béla Szász,1868,Écrivain. - Juge. - Traducteur. - A fait des é...
160756,http://data.bnf.fr/ark:/12148/cb11906728m#about,http://dbpedia.org/resource/Gisèle_Halimi,http://viaf.org/viaf/51690665,Gisèle Halimi,1927,"Avocate. - Femme politique, députée de l'Isère..."
194723,http://data.bnf.fr/ark:/12148/cb12400819r#about,http://dbpedia.org/resource/Claude_Goasguen,http://viaf.org/viaf/44383712,Claude Goasguen,1945,Avocat. - Inspecteur général de l'éducation na...
531196,http://data.bnf.fr/ark:/12148/cb11907611v#about,http://dbpedia.org/resource/Thomas_Hodgskin,,Thomas Hodgskin,1787,"Jounaliste, économiste. - Un des pionniers du ..."
...,...,...,...,...,...,...
162204,http://data.bnf.fr/ark:/12148/cb10272244m#about,http://dbpedia.org/resource/James_Mitchell_(Ca...,http://viaf.org/viaf/51587696,Yiṣḥaq Zelig Gronemann,1843,"Rabbin. - Père de Sammy Gronemann (1875-1952),..."
469623,http://data.bnf.fr/ark:/12148/cb16761971p#about,http://dbpedia.org/resource/Wade_H._McCree,http://viaf.org/viaf/116498531,Veljko Guberina,1925,Avocat
43336,http://data.bnf.fr/ark:/12148/cb121974159#about,http://dbpedia.org/resource/Mitrofan_Grodzitsky,http://viaf.org/viaf/78772873,Franc Miklošič,1813,Docteur en philosophie en 1838 (de l'Universit...
479613,http://data.bnf.fr/ark:/12148/cb12023012k#about,http://dbpedia.org/resource/Wade_H._McCree,http://viaf.org/viaf/109768106,Ljubomir Tadić,1925,"Juriste, philosophe et homme politique. - Prof..."


## Recordlinkage between BnF Data and Wikidata

In [125]:
left_on=["name_bnf", "placeOfBirth_bnf", "placeOfDeath_bnf"]
right_on=["name_wk", "placeOfBirth_wk", "placeOfDeath_wk"]

In [126]:
matched_results = fuzzymatcher.fuzzy_left_join(BnF_Data,
                                            Wikidata,
                                            left_on,
                                            right_on,
                                            left_id_col='uri_bnf',
                                            right_id_col='uri_wk')

In [134]:
cols_bnf_wk= ["best_match_score","id_bnf","uri_bnf","viaf_bnf", "name_bnf", "dateBirth_bnf", "dateDeath_bnf","placeOfBirth_bnf","placeOfDeath_bnf", "bio_bnf", 'id_wk','uri_wk', 'viaf_wk', 'name_wk', 'dateBirth_wk', "dateDeath_wk", "placeOfBirth_wk", "placeOfDeath_wk"]

In [74]:
best_match_bnf_wk=matched_results[cols_bnf_wk].sort_values(by=["best_match_score"], ascending=False).head(10)
best_match_bnf_wk[:1]

NameError: name 'matched_results' is not defined

In [73]:
worse_bnf_wk=matched_results[cols_bnf_wk].sort_values(by=["best_match_score"], ascending=True).head(10)
worse_bnf_wk[:1]

NameError: name 'matched_results' is not defined

In [140]:
matched_bnf_wk=matched_results[cols_bnf_wk].query("best_match_score >= .35").sort_values(
    by=['best_match_score'], ascending=False)
print(len(matched_bnf_wk))
matched_bnf_wk[-10:]

1098


,best_match_score,id_bnf,uri_bnf,viaf_bnf,name_bnf,dateBirth_bnf,dateDeath_bnf,placeOfBirth_bnf,placeOfDeath_bnf,bio_bnf,id_wk,uri_wk,viaf_wk,name_wk,dateBirth_wk,dateDeath_wk,placeOfBirth_wk,placeOfDeath_wk
1017942,0.358156,9855,http://data.bnf.fr/ark:/12148/cb12415111g#about,NaN,Hajime Kawakami,1879,1946.0,NaN,NaN,"Economiste et essayiste marxiste, propagateur ...",1695.0,http://www.wikidata.org/entity/Q1135505,http://viaf.org/viaf/22933447,Hajime Kawakami,1879-10-20T00:00:00Z,1946-01-30T00:00:00Z,Yamaguchi Prefecture,NaN
998868,0.358036,9592,http://data.bnf.fr/ark:/12148/cb104536635#about,NaN,Antonio Scialoja,1817,1877.0,NaN,NaN,"Homme politique, économiste. - Professeur d'éc...",13223.0,http://www.wikidata.org/entity/Q2857453,http://viaf.org/viaf/68918836,Antonio Scialoja,1817-08-01T00:00:00Z,1877-10-13T00:00:00Z,Naples,Procida
130344,0.357927,1264,http://data.bnf.fr/ark:/12148/cb119088558#about,http://viaf.org/viaf/73855183,Adolphe Joanne,1813,1881.0,Dijon,Paris,"Voyageur, créateur de collections de guides gé...",1024.0,http://www.wikidata.org/entity/Q481421,http://viaf.org/viaf/34456999,Jean Gaudemet,1908-09-10T00:00:00Z,2001-05-17T00:00:00Z,Dijon,Paris
279611,0.357148,2786,http://data.bnf.fr/ark:/12148/cb12342406f#about,http://viaf.org/viaf/54220961,Riccardo Monaco,1909,2000.0,"Gênes, Italie",Rome,Juriste. - Ancien professeur de droit internat...,42107.0,http://www.wikidata.org/entity/Q61272919,http://viaf.org/viaf/54220961,Riccardo Monaco,1909-01-01T00:00:00Z,2000-01-18T00:00:00Z,Genoa,Rome
1121140,0.355670,10686,http://data.bnf.fr/ark:/12148/cb10333378s#about,NaN,Fredrik Stang,1867,1941.0,NaN,NaN,Professeur de droit. - Homme politique,18550.0,http://www.wikidata.org/entity/Q5499526,NaN,Fredrik Stang Heffermehl,1913-03-22T00:00:00Z,1993-02-27T00:00:00Z,NaN,NaN
688231,0.355487,6568,http://data.bnf.fr/ark:/12148/cb123451337#about,http://viaf.org/viaf/22884434,August Becher,1816,1890.0,NaN,NaN,Avocat et homme politique allemand. - Député a...,6144.0,http://www.wikidata.org/entity/Q760865,http://viaf.org/viaf/22884434,August Becher,1816-02-21T00:00:00Z,1890-08-11T00:00:00Z,Stuttgart,Stuttgart
663158,0.354356,6335,http://data.bnf.fr/ark:/12148/cb10991687c#about,http://viaf.org/viaf/2462419,Armand Bernard,1868,1935.0,Montbéliard (Doubs),Monte-Carlo,Licencié en droit. - Haut-fonctionnaire. - Avo...,39850.0,http://www.wikidata.org/entity/Q47136452,http://viaf.org/viaf/2462419,Armand Bernard,1868-03-08T00:00:00Z,1935-08-01T00:00:00Z,Montbéliard,NaN
387016,0.352890,3832,http://data.bnf.fr/ark:/12148/cb13036547p#about,http://viaf.org/viaf/41975189,Edmond-Eugène Thaller,1851,1918.0,Husseren-Wesserling (Haut-Rhin),Paris,Professeur de droit commercial à l'Université ...,51267.0,http://www.wikidata.org/entity/Q101995798,http://viaf.org/viaf/41975189,Edmond-Eugène Thaller,1851-01-01T00:00:00Z,1918-01-01T00:00:00Z,NaN,NaN
238123,0.352629,2357,http://data.bnf.fr/ark:/12148/cb13005078q#about,http://viaf.org/viaf/59217659,Frédéric Béchard,1824,1898.0,Nîmes,Neuilly-sur-Seine,Romancier et auteur dramatique. - Avocat (1846...,6149.0,http://www.wikidata.org/entity/Q180409,http://viaf.org/viaf/24606197,Frédéric Passy,1822-05-20T00:00:00Z,1912-06-12T00:00:00Z,Paris,Neuilly-sur-Seine
1005801,0.350575,9681,http://data.bnf.fr/ark:/12148/cb12246037d#about,NaN,Stefan Starzyński,1893,1939.0,Varsovie,Varsovie,Homme politique et économiste. - Maire de la v...,17341.0,http://www.wikidata.org/entity/Q3849484,http://viaf.org/viaf/27872662,Stefan Starzyński,1893-08-19T00:00:00Z,1943-01-01T00:00:00Z,Warsaw,Dachau concentration camp


## Recordlinkage between DBpedia and Wikidata

In [9]:
left_on=["name_dbp", "placeOfBirth_dbp", "placeOfDeath_dbp", "birthDate_dbp" ]
right_on=["name_wk", "placeOfBirth_wk", "placeOfDeath_wk", "dateBirth_wk"]

In [10]:
matched_results = fuzzymatcher.fuzzy_left_join(DBpedia,
                                            Wikidata,
                                            left_on,
                                            right_on,
                                            left_id_col='uri_dbp',
                                            right_id_col='uri_wk')

In [11]:
cols_dbp_wk= ["best_match_score","uri_dbp","viaf_dbp", "name_dbp", "birthDate_dbp", "deathDate_dbp","placeOfBirth_dbp","placeOfDeath_dbp", "uri_wk", "viaf_wk", "name_wk","dateBirth_wk","dateDeath_wk","placeOfBirth_wk","placeOfDeath_wk"]

In [12]:
best_match_dbp_wk=matched_results[cols_dbp_wk].sort_values(by=["best_match_score"], ascending=False).head(10)
best_match_dbp_wk

,best_match_score,uri_dbp,viaf_dbp,name_dbp,birthDate_dbp,deathDate_dbp,placeOfBirth_dbp,placeOfDeath_dbp,uri_wk,viaf_wk,name_wk,dateBirth_wk,dateDeath_wk,placeOfBirth_wk,placeOfDeath_wk
100825,3.217544,http://dbpedia.org/resource/Carl_Ernst_Fürst_F...,http://viaf.org/viaf/47524071,Carl Ernst Fürst Fugger von Glött,1859-07-02,1940-04-25,Oberndorf am Lech,Kirchheim in Schwaben,http://www.wikidata.org/entity/Q18130550,http://viaf.org/viaf/47524071,Carl Ernst Fürst Fugger von Glött,1859-07-02T00:00:00Z,1940-04-25T00:00:00Z,Oberndorf am Lech,Kirchheim in Schwaben
83247,1.963506,http://dbpedia.org/resource/Pieter_Cort_van_de...,http://viaf.org/viaf/55359941,Pieter Cort van der Linden,1846-05-14,1935-07-15,The Hague,The Hague,http://www.wikidata.org/entity/Q1397415,http://viaf.org/viaf/55359941,Pieter Cort van der Linden,1846-05-14T00:00:00Z,1935-07-15T00:00:00Z,The Hague,The Hague
221067,1.844043,http://dbpedia.org/resource/K._N._Raj,http://viaf.org/viaf/110206928,K. N. Raj,1924-05-13,2010-02-10,Thrissur district,Thiruvananthapuram,http://www.wikidata.org/entity/Q6323660,http://viaf.org/viaf/110206928,K. N. Raj,1924-05-13T00:00:00Z,2010-02-10T00:00:00Z,Thrissur district,Thiruvananthapuram
248885,1.744106,http://dbpedia.org/resource/Bento_de_Jesus_Caraça,NaN,Bento de Jesus Caraça,1901-04-18,--06-25,Vila Viçosa,Lisbon,http://www.wikidata.org/entity/Q4890520,http://viaf.org/viaf/32124313,Bento de Jesus Caraça,1901-04-18T00:00:00Z,1948-06-25T00:00:00Z,Vila Viçosa,Lisbon
39107,1.699398,http://dbpedia.org/resource/Peter_S._Albin,http://viaf.org/viaf/73451176,Peter S. Albin,1934-12-20,2008-02-20,New York City,New York City,http://www.wikidata.org/entity/Q4060600,http://viaf.org/viaf/73451176,Peter S. Albin,1934-12-20T00:00:00Z,2008-02-20T00:00:00Z,New York City,New York City
247808,1.699032,http://dbpedia.org/resource/Bagrat_Asatryan,NaN,Bagrat Asatryan,1956-02-02,NaN,Armenian Soviet Socialist Republic,NaN,http://www.wikidata.org/entity/Q4071171,NaN,Bagrat Asatryan,1956-02-02T00:00:00Z,NaN,Armenian Soviet Socialist Republic,NaN
146525,1.678884,http://dbpedia.org/resource/Wouter_Koolmees,http://viaf.org/viaf/290887085,Wouter Koolmees,1977-03-20,NaN,Capelle aan den IJssel,NaN,http://www.wikidata.org/entity/Q2595414,http://viaf.org/viaf/290887085,Wouter Koolmees,1977-03-20T00:00:00Z,NaN,Capelle aan den IJssel,NaN
63114,1.656931,http://dbpedia.org/resource/Oscar_Espinosa_Chepe,http://viaf.org/viaf/64254939,Oscar Espinosa Chepe,1940-11-29,2013-09-23,Cienfuegos,Cercedilla,http://www.wikidata.org/entity/Q7105959,http://viaf.org/viaf/64254939,Oscar Espinosa Chepe,1940-11-29T00:00:00Z,2013-09-23T00:00:00Z,Cienfuegos,Cercedilla
359358,1.611539,http://dbpedia.org/resource/Mariano_Rajoy_Sobredo,NaN,Mariano Rajoy Sobredo,1921-08-28,2018-11-01,Santiago de Compostela,Madrid,http://www.wikidata.org/entity/Q22122581,NaN,Mariano Rajoy Sobredo,1921-08-28T00:00:00Z,2018-11-01T00:00:00Z,Santiago de Compostela,Madrid
110938,1.600508,http://dbpedia.org/resource/Justin_Wolfers,http://viaf.org/viaf/42892681,Justin Wolfers,1972-12-11,NaN,Papua New Guinea,NaN,http://www.wikidata.org/entity/Q1714242,http://viaf.org/viaf/42892681,Justin Wolfers,1972-12-11T00:00:00Z,NaN,Papua New Guinea,NaN


In [13]:
worse_dbp_wk=matched_results[cols_dbp_wk].sort_values(by=["best_match_score"], ascending=True).head(10)
worse_dbp_wk

,best_match_score,uri_dbp,viaf_dbp,name_dbp,birthDate_dbp,deathDate_dbp,placeOfBirth_dbp,placeOfDeath_dbp,uri_wk,viaf_wk,name_wk,dateBirth_wk,dateDeath_wk,placeOfBirth_wk,placeOfDeath_wk
559316,-1.729896,http://dbpedia.org/resource/Gilberto_Concepció...,NaN,Gilberto Concepción de Gracia,1909-07-09,1968-03-16,"Vega Alta, Puerto Rico","Santurce, San Juan, Puerto Rico",http://www.wikidata.org/entity/Q5561403,NaN,"Gilberto Teodoro, Sr.",1927-05-07T00:00:00Z,2008-02-22T00:00:00Z,NaN,NaN
154237,-1.704865,http://dbpedia.org/resource/C._Hayavadana_Rao,http://viaf.org/viaf/273110043,http://dbpedia.org/resource/Rao_Bahadur,1865-07-10,1946-01-27,Presidencies and provinces of British India,Kingdom of Mysore,http://www.wikidata.org/entity/Q4890148,http://viaf.org/viaf/95158137,Benoy Kumar Sarkar,1887-12-26T00:00:00Z,1949-11-24T00:00:00Z,Provinces of India,"Washington, D.C."
390291,-1.656593,http://dbpedia.org/resource/Vernon_Geddy,NaN,http://dbpedia.org/resource/William_&_Mary_Tri...,1897-11-11,1952-10-18,"Williamsburg, Virginia","James City County, Virginia",http://www.wikidata.org/entity/Q25483624,NaN,NaN,1880-11-14T00:00:00Z,1932-01-01T00:00:00Z,NaN,Granite City
323713,-1.637836,http://dbpedia.org/resource/Byron_B._Harlan,NaN,Byron Berry Harlan,1886-10-22,1949-11-11,"Greenville, Ohio","Cogan House Township, Lycoming County, Pennsyl...",http://www.wikidata.org/entity/Q5075578,http://viaf.org/viaf/25845506,Charles Berry,1930-01-01T00:00:00Z,2007-01-01T00:00:00Z,NaN,NaN
697509,-1.499284,http://dbpedia.org/resource/Kinahan_Cornwallis...,NaN,Kinahan Cornwallis [Albert MacKenzie Russell K...,1837-12-24,--08-15,United Kingdom of Great Britain and Ireland,NaN,http://www.wikidata.org/entity/Q7381844,http://viaf.org/viaf/85295959,Russell W. Cooper,1955-01-01T00:00:00Z,NaN,United States of America,NaN
176915,-1.492256,http://dbpedia.org/resource/Asta_Hampe,http://viaf.org/viaf/213397360,Prof. Dr. rer.pol. und Dipl.-Ing. Asta Hampe,1907-05-24,2003-10-22,Helmstedt,Hamburg,http://www.wikidata.org/entity/Q96178,http://viaf.org/viaf/30485602,Emil Puhl,1889-08-28T00:00:00Z,1962-03-30T00:00:00Z,Berlin,Hamburg
555251,-1.484274,http://dbpedia.org/resource/Clément-Charles_Sa...,NaN,Clément-Charles Sabrevois de Bleury,1798-10-28,1862-09-15,Lower Canada,"Saint-Vincent-de-Paul, Quebec",http://www.wikidata.org/entity/Q2958324,http://viaf.org/viaf/194159830,Charles-Victor de Bavay,1801-01-01T00:00:00Z,1875-11-28T00:00:00Z,NaN,NaN
231567,-1.479503,http://dbpedia.org/resource/Giuseppe_Toniolo,http://viaf.org/viaf/10642260,http://dbpedia.org/resource/Beatification,1845-03-07,1918-10-07,Kingdom of Lombardy–Venetia,Kingdom of Italy,http://www.wikidata.org/entity/Q12360110,NaN,Avo Org,1952-03-23T00:00:00Z,NaN,NaN,NaN
463185,-1.414708,http://dbpedia.org/resource/Charles_A._Eldredge,NaN,Charles A. Eldredge,1820-02-27,1896-10-26,"Bridport, Vermont","Fond du Lac, Wisconsin",http://www.wikidata.org/entity/Q5234631,http://viaf.org/viaf/31429379,David Hale,1951-11-22T00:00:00Z,2015-10-19T00:00:00Z,Vermont,NaN
549970,-1.414587,http://dbpedia.org/resource/Caleb_Dorsey,NaN,Caleb Dorsey,1825-05-25,1885-03-29,"Anne Arundel County, Maryland","Sonora, California",http://www.wikidata.org/entity/Q5109543,NaN,Christian Dorsey,1971-01-01T00:00:00Z,NaN,Atlantic City,NaN


In [22]:
matched_dbp_wk=matched_results[cols_dbp_wk].query("best_match_score >= .15").sort_values(
    by=['best_match_score'], ascending=False)
print(len(matched_dbp_wk))
matched_dbp_wk[-10:]

1255


,best_match_score,uri_dbp,viaf_dbp,name_dbp,birthDate_dbp,deathDate_dbp,placeOfBirth_dbp,placeOfDeath_dbp,uri_wk,viaf_wk,name_wk,dateBirth_wk,dateDeath_wk,placeOfBirth_wk,placeOfDeath_wk
253927,0.154060,http://dbpedia.org/resource/Ken_Ofori-Atta,NaN,Kenneth Ofori-Atta,1958-11-07,NaN,"Kibi, Ghana",NaN,http://www.wikidata.org/entity/Q20022251,NaN,Ken Ofori-Atta,1959-01-01T00:00:00Z,NaN,Kibi,NaN
281870,0.154006,http://dbpedia.org/resource/Maria_Kiwanuka,NaN,Maria Kiwanuka,1955-05-12,NaN,Uganda,NaN,http://www.wikidata.org/entity/Q6761316,NaN,Maria Kiwanuka,1955-05-12T00:00:00Z,NaN,Kampala,NaN
224324,0.153920,http://dbpedia.org/resource/Heinz_Arndt,http://viaf.org/viaf/108960473,Heinz Arndt,1915-02-26,2002-04-06,NaN,NaN,http://www.wikidata.org/entity/Q5700560,http://viaf.org/viaf/108960473,Heinz Arndt,1915-02-26T00:00:00Z,2002-05-06T00:00:00Z,Wrocław,Canberra
276498,0.153733,http://dbpedia.org/resource/Pilar_Nores_de_García,NaN,Pilar Nores,1949-03-11,NaN,Argentina,NaN,http://www.wikidata.org/entity/Q9059638,NaN,Pilar Nores de García,1949-03-11T00:00:00Z,NaN,Córdoba,NaN
10116,0.153416,http://dbpedia.org/resource/Laurence_Meyer,http://viaf.org/viaf/91708741,Laurence Meyer,1944-03-08,NaN,NaN,NaN,http://www.wikidata.org/entity/Q6500730,http://viaf.org/viaf/91708741,Laurence Meyer,1944-03-08T00:00:00Z,NaN,The Bronx,NaN
139403,0.153068,http://dbpedia.org/resource/Martín_Guzmán,http://viaf.org/viaf/306384144,Martín Guzmán,1982-10-12,NaN,Argentina,NaN,http://www.wikidata.org/entity/Q77605455,http://viaf.org/viaf/306384144,Martín Guzmán,1982-10-12T00:00:00Z,NaN,La Plata,NaN
107028,0.152550,http://dbpedia.org/resource/Laura_Tyson,http://viaf.org/viaf/44397217,Laura Tyson,1947-06-28,NaN,United States,NaN,http://www.wikidata.org/entity/Q460911,http://viaf.org/viaf/44397217,Laura Tyson,1947-06-28T00:00:00Z,NaN,Bayonne,NaN
280497,0.151896,http://dbpedia.org/resource/Edward_Sandoyan,NaN,Edward Sandoyan,1961-06-04,NaN,Soviet Union,NaN,http://www.wikidata.org/entity/Q4407546,NaN,Edward Sandoyan,1961-06-04T00:00:00Z,NaN,Yerevan,NaN
15852,0.150671,http://dbpedia.org/resource/Manuel_H._Johnson,http://viaf.org/viaf/8678224,Manley Johnson,1949-02-10,NaN,NaN,NaN,http://www.wikidata.org/entity/Q3286848,http://viaf.org/viaf/8678224,Manuel H. Johnson,1949-02-10T00:00:00Z,NaN,Troy,NaN
43492,0.150061,http://dbpedia.org/resource/Thorstein_Veblen,http://viaf.org/viaf/71451539,NaN,1857-07-30,1929-08-03,NaN,NaN,http://www.wikidata.org/entity/Q4105702,NaN,NaN,1857-07-22T00:00:00Z,1927-03-16T00:00:00Z,NaN,NaN


In [21]:
left_on=["name_bnf"]
right_on=["name_wk"]

In [22]:
matched_results = fuzzymatcher.fuzzy_left_join(BnF_Data,
                                            Wikidata,
                                            left_on,
                                            right_on,
                                            left_id_col='uri_bnf',
                                            right_id_col='uri_wk')

In [34]:
cols_bnf_wk= ["best_match_score","uri_bnf","viaf_bnf", "name_bnf", "yearBirth_bnf", "yearDeath_bnf","placeOfBirth_bnf","placeOfDeath_bnf","bio_bnf", "uri_wk", "viaf_wk", "name_wk","yearBirth_wk","yearDeath_wk","placeOfBirth_wk","placeOfDeath_wk"]

In [49]:
best_match_bnf_wk=matched_results[cols_bnf_wk].sort_values(by=["best_match_score"], ascending=True).head(10)
best_match_bnf_wk

,best_match_score,uri_bnf,viaf_bnf,name_bnf,yearBirth_bnf,yearDeath_bnf,placeOfBirth_bnf,placeOfDeath_bnf,bio_bnf,uri_wk,viaf_wk,name_wk,yearBirth_wk,yearDeath_wk,placeOfBirth_wk,placeOfDeath_wk
429712,-1.702404,http://data.bnf.fr/ark:/12148/cb17791066c#about,NaN,Sayf ibn Ḥamad ibn Šayẖān ibn Muḥammad ibn ...,1892,1961.0,"Sima, Azki - Oman","Matrah, Mascate - Oman",Poète. - Juge,http://www.wikidata.org/entity/Q60576073,http://viaf.org/viaf/47994464,Muhammad al-Imadi,1930.0,NaN,Damascus,NaN
365152,-0.824534,http://data.bnf.fr/ark:/12148/cb12090818k#about,NaN,Federigo Melis,1914,1973.0,"Florence, Italie","Florence, Italie",Historien et économiste. - Professeur d'histoi...,http://www.wikidata.org/entity/Q15879146,NaN,Marie Jacob Hendrik de Bruyn van Melis- en Mar...,1891.0,1964.0,NaN,NaN
152012,-0.690152,http://data.bnf.fr/ark:/12148/cb12536971c#about,http://viaf.org/viaf/39489194,ʿAbd al-Hādī ibn Muḥammad Bū-Ṭālib,1923,2009.0,NaN,NaN,Homme politique. - Professeur de droit constit...,http://www.wikidata.org/entity/Q770901,http://viaf.org/viaf/34529103,Richard T. Ely,1854.0,1943.0,Ripley,Old Lyme
114667,-0.631786,http://data.bnf.fr/ark:/12148/cb10272244m#about,http://viaf.org/viaf/51587696,Yiṣḥaq Zelig Gronemann,1843,1918.0,"Flötenstein (Prusse-Occidentale), aujourd'hui ...",Hannover (Allemagne),"Rabbin. - Père de Sammy Gronemann (1875-1952),...",http://www.wikidata.org/entity/Q2157418,http://viaf.org/viaf/86448277,Robert H. Frank,1945.0,NaN,NaN,NaN
274785,-0.622651,http://data.bnf.fr/ark:/12148/cb16732284q#about,http://viaf.org/viaf/21154368,Yūsuf Hammām Āṣāf,1859,1938.0,NaN,NaN,Historie et homme de lettres. - A été juriste ...,http://www.wikidata.org/entity/Q100989335,NaN,Nils af Björksten,1931.0,NaN,NaN,NaN
220666,-0.621390,http://data.bnf.fr/ark:/12148/cb10099705v#about,http://viaf.org/viaf/286365153,ʿAbd al-qādir ibn Muṣṭafā al- Maġribī,1867,1956.0,Lattaquié,NaN,Juriste. - Journaliste. - A enseigné la langue...,http://www.wikidata.org/entity/Q770901,http://viaf.org/viaf/34529103,Richard T. Ely,1854.0,1943.0,Ripley,Old Lyme
317240,-0.586223,http://data.bnf.fr/ark:/12148/cb14552345w#about,http://viaf.org/viaf/15012801,ʿAzīz Muḥammad Abāẓaẗ,1899,1973.0,Muhafazat al-Charqiyya (Egypte),NaN,Poète et dramaturge. - Avocat. - Député,http://www.wikidata.org/entity/Q4225600,http://viaf.org/viaf/62135397,M. M. Kovalev,1947.0,NaN,Dokšycy District,NaN
115758,-0.585637,http://data.bnf.fr/ark:/12148/cb136243015#about,http://viaf.org/viaf/5109006,Menaḥem Zevi Kaddari,1925,2011.0,Mezőkövesd (Hongrie),Ramat-Gan (Israël),"Linguiste. - Professeur émérite, Department of...",http://www.wikidata.org/entity/Q94878988,http://viaf.org/viaf/57363085,Ernst Am Ende,1804.0,1876.0,NaN,NaN
192591,-0.569673,http://data.bnf.fr/ark:/12148/cb10372967c#about,http://viaf.org/viaf/314851273,Jeanin Roustan,1846,1886.0,Gordes (Vaucluse),Nice (Alpes-Maritimes),Avocat à la Cour d'appel de Lyon. - Prénoms co...,http://www.wikidata.org/entity/Q42257717,http://viaf.org/viaf/94355869,Ann Friedlaender,1938.0,1992.0,Philadelphia,Boston
348242,-0.569062,http://data.bnf.fr/ark:/12148/cb128832222#about,http://viaf.org/viaf/108624624,Muḥammad Ẓafr Allāh H̱ān,1893,1985.0,Sialkot (Pakistan),Lahore (Pakistan),"Juriste, diplomate et homme politique",http://www.wikidata.org/entity/Q60576073,http://viaf.org/viaf/47994464,Muhammad al-Imadi,1930.0,NaN,Damascus,NaN


 ## Second Method: Recordlinkage

In [573]:
BnF_Data = pd.read_csv('df_bnf.csv', index_col='id_bnf')
Wikidata = pd.read_csv('df_wk.csv', index_col='id_wk')
DBpedia = pd.read_csv('df_dbp.csv', index_col='id_dbp')

In [574]:
import recordlinkage
# You can read the documentation https://recordlinkage.readthedocs.io/en/latest/about.html 

--------------------
## Matched BnF Data and Wikidata

In [575]:
indexer = recordlinkage.Index()
indexer.sortedneighbourhood(left_on='name_bnf', right_on='name_wk')
candidates = indexer.index(BnF_Data, Wikidata)
print(len(candidates))

14131


indexer = recordlinkage.Index()
indexer.block(left_on=['name_bnf', 'uri_bnf'],
              right_on=['name_dbp', 'uri_dbp'])
pairs = indexer.index(df_bnf, df_dbp)


candidates = indexer.index(BnF_Data, DBpedia)
print(len(candidates))

In [576]:
compare = recordlinkage.Compare()
compare.string('name_bnf',
            'name_wk',
            method='jarowinkler',
            threshold=0.85,
            label='name_bnf_wk')
compare.exact('dateBirth_bnf',
            'dateBirth_wk',
            label='birthDate_bnf_wk')
compare.exact('dateDeath_bnf',
            'dateDeath_wk',
            label='deathDate_bnf_wk')
features = compare.compute(candidates, BnF_Data, Wikidata)

In [577]:
features.sum(axis=1).value_counts().sort_index(ascending=False)

3.0     598
2.0     418
1.0    6768
0.0    6347
dtype: int64

In [578]:
features

,,name_bnf_wk,birthDate_bnf_wk,deathDate_bnf_wk
id_bnf,id_wk,,,
2,12248,1.0,0,0
4,14978,0.0,0,0
6,3893,1.0,0,0
7,35317,0.0,0,0
8,12837,0.0,0,0
...,...,...,...,...
11125,41767,1.0,0,0
11127,44289,0.0,0,0
11128,38487,1.0,0,0


In [579]:
potential_matches = features[features.sum(axis=1) >1].reset_index()
potential_matches['Score'] = potential_matches.loc[:, 'name_bnf_wk':'deathDate_bnf_wk'].sum(axis=1)
potential_matches

,id_bnf,id_wk,name_bnf_wk,birthDate_bnf_wk,deathDate_bnf_wk,Score
0,274,7028,1.0,1,1,3.0
1,529,15865,1.0,1,1,3.0
2,696,881,1.0,1,1,3.0
3,758,14054,1.0,1,1,3.0
4,1741,1224,1.0,1,1,3.0
...,...,...,...,...,...,...
1011,10220,16302,1.0,1,0,2.0
1012,10373,45470,1.0,0,1,2.0
1013,11083,18223,1.0,1,1,3.0
1014,11116,53779,1.0,1,1,3.0


In [580]:
Wikidata.loc[35036,:]

uri_wk                    http://www.wikidata.org/entity/Q23150294
name_wk                                                Emil Sobota
dateBirth_wk                                            1892-05-17
dateDeath_wk                                            1945-04-14
placeOfBirth_wk                                             Beroun
placeOfDeath_wk                                             Prague
viaf                   http://viaf.org/viaf/7140151248006344270009
uri_bnf            http://data.bnf.fr/ark:/12148/cb12976597g#about
Name: 35036, dtype: object

In [581]:
BnF_Data.loc[5619,:]

uri_bnf             http://data.bnf.fr/ark:/12148/cb12958717m#about
viaf                                 http://viaf.org/viaf/286811086
name_bnf                                                Emil Sobota
Sname                                                           NaN
dateBirth_bnf                                            1892-05-17
dateDeath_bnf                                            1945-04-24
placeOfBirth_bnf       Beroun (Bohême-Centrale, République tchèque)
placeOfDeath_bnf                        Prague (République tchèque)
bio_bnf                  Docteur en droit. - Conseiller ministériel
Name: 5619, dtype: object

In [582]:
Wikidata['name_wk']=Wikidata['name_wk'].astype(str)
BnF_Data['name_bnf']=BnF_Data['name_bnf'].astype(str)

Wikidata['viaf']=Wikidata['viaf'].astype(str)
BnF_Data['viaf']=BnF_Data['viaf'].astype(str)

Wikidata['uri_wk']=Wikidata['uri_wk'].astype(str)
BnF_Data['uri_bnf']=BnF_Data['uri_bnf'].astype(str)

Wikidata['placeOfBirth_wk']=Wikidata['placeOfBirth_wk'].astype(str)
BnF_Data['placeOfBirth_bnf']=BnF_Data['placeOfBirth_bnf'].astype(str)

Wikidata['placeOfDeath_wk']=Wikidata['placeOfDeath_wk'].astype(str)
BnF_Data['placeOfDeath_bnf']=BnF_Data['placeOfDeath_bnf'].astype(str)

BnF_Data['dateDeath_bnf']=BnF_Data['dateDeath_bnf'].astype(str)
Wikidata['dateDeath_wk']=Wikidata['dateDeath_wk'].astype(str)

BnF_Data['dateBirth_bnf']=BnF_Data['dateBirth_bnf'].astype(str)
Wikidata['dateBirth_wk']=Wikidata['dateBirth_wk'].astype(str)

BnF_Data['bio_bnf']=BnF_Data['bio_bnf'].astype(str)

In [583]:
Wikidata['Wikidata_Name_Lookup'] = Wikidata[[
   'name_wk', 'dateBirth_wk','dateDeath_wk','placeOfBirth_wk' ,'placeOfDeath_wk' 
]].apply(lambda x: '|'.join(x), axis=1)

BnF_Data['bnf_Name_Lookup'] = BnF_Data[[
   'name_bnf','dateBirth_bnf','dateDeath_bnf','placeOfBirth_bnf','placeOfDeath_bnf','bio_bnf' 
]].apply(lambda x: '|'.join(x), axis=1)

Wikidata_lookup = Wikidata[['Wikidata_Name_Lookup']].reset_index()
BnF_Data_lookup = BnF_Data[['bnf_Name_Lookup']].reset_index()

In [584]:
Wikidata_merge = potential_matches.merge(Wikidata_lookup, how='left')

In [585]:
final_wk_bnf_merge = Wikidata_merge.merge(BnF_Data_lookup, how='left')

In [586]:
cols = ['id_bnf', 'id_wk', 'Score',
        'bnf_Name_Lookup', 'Wikidata_Name_Lookup']
final_wk_bnf_merge=final_wk_bnf_merge[cols].sort_values(by=[ 'Score'], ascending=True)
print(len(final_wk_bnf_merge))
final_wk_bnf_merge[:50]

1016


,id_bnf,id_wk,Score,bnf_Name_Lookup,Wikidata_Name_Lookup
507,5492,27548,2.0,Yvon Linant de Bellefonds|1904-08-27|1994-12-2...,Yvon Linant de Bellefonds|1904-01-01|1994-12-2...
520,5818,26807,2.0,Federico Patellani|1911-12-01|1977-02-10|Monza...,Federico Patellani|1911-12-01|1977-01-01|Monza...
522,5887,30520,2.0,Charles Lyon-Caen|1843-12-25|1935-09-17|Paris ...,Charles Lyon-Caen|1843-12-25|1935-12-17|Paris|...
523,5889,2375,2.0,Charles-Frédéric Rau|1803-08-03|1877-04-10|Sav...,Charles-Frédéric Rau|1803-08-03|1877-04-1|Boux...
525,5897,10622,2.0,Robert Walter|1931-01-30|2010-12-25|nan|nan|Ju...,Robert Walter|1931-01-3|2010-12-25|Vienna|Vienna
526,5899,26491,2.0,Salvatore Riccobono|1864-01-31|1958-04-05|San ...,Salvatore Riccobono|1864-01-31|1958-04-12|San ...
527,5901,52871,2.0,Roman Herzog|1934-04-05|2017-01-10|Landshut (B...,Roman Herzog|1934-04-05|2017-01-1|Landshut|Bad...
518,5729,97,2.0,Hugo Sinzheimer|1875-04-12|1945-09-16|Worms (R...,Hugo Sinzheimer|1875-01-01|1945-09-16|Worms|Bl...
528,5905,3888,2.0,Ottó Bihari|1921-01-11|1983-01-04|Timis̡oara (...,Ottó Bihari|1921-01-13|1983-01-04|Timișoara|Pécs
533,5976,41022,2.0,Otto Weissel|1875-08-31|1955|Vienne|nan|Avocat,Otto Weissel|1875-08-31|1955-12-23|Vienna|Geneva


In [587]:
# We realize a merge between Wikidata and BnF Data using the BnF URIs in common
merged_final_bnf_wk = merged_bnf_wk.merge(final_wk_bnf_merge, on=['id_wk', 'id_bnf'], how='outer')
print("")
print("This method merge", len(merged_final_bnf_wk), "persons.")
print("")
print(len(merged_final_bnf_wk)-len(merged_bnf_wk), "persons have not a BnF URI in common on Wikidata.")
print("")
cols=["id_bnf","id_wk","uri_bnf","uri_wk", "name_bnf", "dateBirth_bnf","dateDeath_bnf","placeOfBirth_bnf","placeOfDeath_bnf","bio_bnf","name_wk","dateBirth_wk","dateDeath_wk","placeOfBirth_wk","placeOfDeath_wk"]
merged_final_bnf_wk[cols][:10]


This method merge 1597 persons.

39 persons have not a BnF URI in common on Wikidata.



,id_bnf,id_wk,uri_bnf,uri_wk,name_bnf,dateBirth_bnf,dateDeath_bnf,placeOfBirth_bnf,placeOfDeath_bnf,bio_bnf,name_wk,dateBirth_wk,dateDeath_wk,placeOfBirth_wk,placeOfDeath_wk
0,1713,6505,http://data.bnf.fr/ark:/12148/cb10071436z#about,http://www.wikidata.org/entity/Q1360518,Ernst Walz,1859-07-18,1941-12-18,"Heidelberg (Bade-Wurtemberg, Allemagne)","Heidelberg (Bade-Wurtemberg, Allemagne)","Maire de Heidelberg, Allemagne (1886-). - A ét...",Ernst Walz,1859-07-19,1941-12-18,Heidelberg,Heidelberg
1,6594,28290,http://data.bnf.fr/ark:/12148/cb101827728#about,http://www.wikidata.org/entity/Q15177628,Adrien Calmètes,1800-09-19,1871-02-27,Figueras (Espagne),Montpellier (Hérault),Magistrat. - Président de chambre à Montpellie...,Adrien Calmètes,1800-09-19,1871-02-27,Figueres,Montpellier
2,6600,44446,http://data.bnf.fr/ark:/12148/cb10190862z#about,http://www.wikidata.org/entity/Q56646471,Béla Kun,1861-04-24,1934-09-19,"Sátoraljaújhelyi, Hongrie","Budapest, Hongrie",Juriste. - Conseiller au Ministère de la Justi...,Kun Béla,1861-04-24,1934-09-19,Sátoraljaújhely,Budapest District VII
3,6929,6082,http://data.bnf.fr/ark:/12148/cb102042826#about,http://www.wikidata.org/entity/Q1214240,Imre Nagy,1822-07-01,1894-05-05,"Németkeresztúr, aujourd'hui Deutschkreutz, Aut...",Budapest,Historien. - Juriste. - Académicien,Imre Nagy,1822-06-01,1894-05-05,Deutschkreutz,Budapest
4,6931,46637,http://data.bnf.fr/ark:/12148/cb10207440j#about,http://www.wikidata.org/entity/Q94851244,Wilhelm Gustav Karl Starke,1824-02-26,1903-03-10,"Lubán (Prusse, aujourd'hui Pologne)",Berlin (Allemagne),Juriste. - Parlementaire,Wilhelm Gustav Karl Starke,1824-02-26,1903-03-09,Lubań,Berlin
5,7694,2149,http://data.bnf.fr/ark:/12148/cb102101229#about,http://www.wikidata.org/entity/Q343248,Karel Baxa,1863-06-23,1938-01-05,NaN,NaN,Docteur en droit. - Avocat. - Membre du parti ...,Karel Baxa,1862-06-24,1938-01-05,Sedlčany,Prague
6,3976,28075,http://data.bnf.fr/ark:/12148/cb10217219c#about,http://www.wikidata.org/entity/Q15830136,Jan Heller,1848-11-13,1932-03-20,Vranov u Rokycan (République tchèque),Prague (République tchèque),"Juriste, publiciste. - Rédacteur de la revue j...",Jan Heller,1848-11-13,1932-03-2,NaN,Prague
7,1011,15408,http://data.bnf.fr/ark:/12148/cb102252805#about,http://www.wikidata.org/entity/Q3840074,Luigi Rava,1860-12-01,1938-05-12,"Ravenne, Italie",Rome,"Juriste et historien, professeur de philosophi...",Luigi Rava,1860-11-29,1938-05-12,Ravenna,Rome
8,7407,45126,http://data.bnf.fr/ark:/12148/cb10226211k#about,http://www.wikidata.org/entity/Q57202321,André Roux,1893-10-10,19..,Privas (Ardèche),NaN,Magistrat. - A été avocat à Montpellier (1915-...,André Louis Roux,1893-10-1,NaN,Privas,NaN
9,10383,29226,http://data.bnf.fr/ark:/12148/cb10226945t#about,http://www.wikidata.org/entity/Q13411514,Arveds Švābe,1888-05-25,1959-08-20,NaN,NaN,"Historien, juriste et auteur",Arveds Švābe,1888-05-25,1959-08-2,Q16362410,Stockholm


## Match between BnF Data and DBpedia

indexer = recordlinkage.Index()
indexer.full()

In [588]:
indexer = recordlinkage.Index()
indexer.sortedneighbourhood(left_on='name_bnf', right_on='name_dbp')
candidates = indexer.index(BnF_Data, DBpedia)
print(len(candidates))

4266


indexer = recordlinkage.Index()
indexer.block(left_on=['name_bnf', 'uri_bnf'],
              right_on=['name_dbp', 'uri_dbp'])
pairs = indexer.index(df_bnf, df_dbp)


candidates = indexer.index(BnF_Data, DBpedia)
print(len(candidates))

In [589]:
compare = recordlinkage.Compare()
compare.string('name_bnf',
            'name_dbp',
            method='jarowinkler',
            threshold=0.85,
            label='name_bnf_dbp')
compare.exact('dateBirth_bnf',
            'birthDate_dbp',
            label='birthDate_bnf_dbp')
compare.exact('dateDeath_bnf',
            'deathDate_dbp',
            label='deathDate_bnf_dbp')
features = compare.compute(candidates, BnF_Data, DBpedia)

In [590]:
features

,,name_bnf_dbp,birthDate_bnf_dbp,deathDate_bnf_dbp
id_bnf,id_dbp,,,
18,6120,0.0,0,0
20,6013,0.0,0,0
21,307,0.0,0,0
25,4316,1.0,0,0
26,4575,0.0,0,0
...,...,...,...,...
11119,6209,0.0,0,0
11120,7073,0.0,0,0
11124,6264,0.0,0,0


In [591]:
features.sum(axis=1).value_counts().sort_index(ascending=False)

3.0      93
2.0      24
1.0    1172
0.0    2977
dtype: int64

In [592]:
features[features.sum(axis=1) > 1]

,,name_bnf_dbp,birthDate_bnf_dbp,deathDate_bnf_dbp
id_bnf,id_dbp,,,
2110,9566,1.0,0,1
5555,8405,1.0,1,0
8876,8224,1.0,1,1
9459,210,1.0,1,1
9597,994,1.0,1,1
...,...,...,...,...
10651,943,1.0,1,0
1643,869,1.0,1,1
6734,3361,1.0,1,1


In [593]:

potential_matches = features[features.sum(axis=1) > 1].reset_index()
potential_matches['Score'] = potential_matches.loc[:, 'name_bnf_dbp':'deathDate_bnf_dbp'].sum(axis=1)
potential_matches
# Display the matches up to 

,id_bnf,id_dbp,name_bnf_dbp,birthDate_bnf_dbp,deathDate_bnf_dbp,Score
0,2110,9566,1.0,0,1,2.0
1,5555,8405,1.0,1,0,2.0
2,8876,8224,1.0,1,1,3.0
3,9459,210,1.0,1,1,3.0
4,9597,994,1.0,1,1,3.0
...,...,...,...,...,...,...
112,10651,943,1.0,1,0,2.0
113,1643,869,1.0,1,1,3.0
114,6734,3361,1.0,1,1,3.0
115,7257,9838,1.0,1,1,3.0


In [594]:
BnF_Data.loc[2110,:]

uri_bnf               http://data.bnf.fr/ark:/12148/cb11043700s#about
viaf                                    http://viaf.org/viaf/62455068
name_bnf                                           Mihai A. Antonescu
Sname                                                             NaN
dateBirth_bnf                                              1907-11-18
dateDeath_bnf                                              1946-06-01
placeOfBirth_bnf                                     Nucet (Roumanie)
placeOfDeath_bnf                                    Jilava (Roumanie)
bio_bnf             Juriste. - Homme politique. - Vice-président d...
bnf_Name_Lookup     Mihai A. Antonescu|1907-11-18|1946-06-01|Nucet...
Name: 2110, dtype: object

In [595]:
DBpedia.loc[9566,:]

uri_dbp             http://dbpedia.org/resource/Mihai_Antonescu
viaf                                                        NaN
name_dbp                                        Mihai Antonescu
birthDate_dbp                                        1904-11-18
deathDate_dbp                                        1946-06-01
placeOfBirth_dbp                             Kingdom of Romania
placeOfDeath_dbp                             Kingdom of Romania
Name: 9566, dtype: object

In [596]:
BnF_merge = potential_matches.merge(BnF_Data, how='left', on="id_bnf")

In [597]:
final_bnf_dbp_merge = BnF_merge.merge(DBpedia, how='left', on='id_dbp')

In [601]:
cols = ["id_bnf","id_dbp","uri_bnf","uri_dbp","name_bnf","dateBirth_bnf","dateDeath_bnf","placeOfBirth_bnf", "placeOfDeath_bnf", "name_dbp", "birthDate_dbp","deathDate_dbp","placeOfBirth_dbp","placeOfDeath_dbp"]
final_bnf_dbp_merge=final_bnf_dbp_merge[cols]
print(len(final_bnf_dbp_merge))
final_bnf_dbp_merge

117


,id_bnf,id_dbp,uri_bnf,uri_dbp,name_bnf,dateBirth_bnf,dateDeath_bnf,placeOfBirth_bnf,placeOfDeath_bnf,name_dbp,birthDate_dbp,deathDate_dbp,placeOfBirth_dbp,placeOfDeath_dbp
0,2110,9566,http://data.bnf.fr/ark:/12148/cb11043700s#about,http://dbpedia.org/resource/Mihai_Antonescu,Mihai A. Antonescu,1907-11-18,1946-06-01,Nucet (Roumanie),Jilava (Roumanie),Mihai Antonescu,1904-11-18,1946-06-01,Kingdom of Romania,Kingdom of Romania
1,5555,8405,http://data.bnf.fr/ark:/12148/cb156079769#about,http://dbpedia.org/resource/John_J._Bennett_Jr.,John James Bennett,1894-03-02,1967-10,Brooklyn (N.Y.),Brooklyn (N.Y.),"John James Bennett, Jr.",1894-03-02,1967-10-04,Brooklyn,Brooklyn
2,8876,8224,http://data.bnf.fr/ark:/12148/cb119433785#about,http://dbpedia.org/resource/Chittaranjan_Das,Chitta Ranjan Das,1870-11-05,1925-06-16,"Bikrampur, Dhaka (maintenant au Bangladesh)",Darjeeling,Chittaranjan Das,1870-11-05,1925-06-16,Presidencies and provinces of British India,NaN
3,9459,210,http://data.bnf.fr/ark:/12148/cb12279046q#about,http://dbpedia.org/resource/Hollis_B._Chenery,Hollis Burnley Chenery,1918-01-06,1994-09-01,"Richmond, Va.","Santa Fe, N. Mex.",Hollis Chenery,1918-01-06,1994-09-01,United States,United States
4,9597,994,http://data.bnf.fr/ark:/12148/cb12277816h#about,http://dbpedia.org/resource/Bertil_Ohlin,Bertil Gotthard Ohlin,1899-04-23,1979-08-03,"Klippan, Suède","Åre, Suède",Bertil Ohlin,1899-04-23,1979-08-03,"Klippan, Scania",Åre Municipality
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
112,10651,943,http://data.bnf.fr/ark:/12148/cb177856643#about,http://dbpedia.org/resource/Laxmi_Mall_Singhvi,Laxmi Mall Singhvi,1931-11-09,2007-10-13,nan,nan,Laxmi Mall Singhvi,1931-11-09,2007-10-06,Presidencies and provinces of British India,India
113,1643,869,http://data.bnf.fr/ark:/12148/cb12054658d#about,http://dbpedia.org/resource/Helmuth_James_von_...,Helmuth James von Moltke,1907-03-11,1945-01-23,nan,nan,Helmuth James Graf von Moltke,1907-03-11,1945-01-23,NaN,Plötzensee Prison
114,6734,3361,http://data.bnf.fr/ark:/12148/cb12380018p#about,http://dbpedia.org/resource/Otto_Lenel,Otto Lenel,1849-12-13,1935-02-07,"Mannheim, Allemagne","Fribourg-en-Brisgau, Allemagne",Otto Lene,1849-12-13,1935-02-07,Grand Duchy of Baden,Freiburg im Breisgau
115,7257,9838,http://data.bnf.fr/ark:/12148/cb12282365b#about,http://dbpedia.org/resource/Lewis_H._Morgan,Lewis Henry Morgan,1818-11-21,1881-12-17,Aurora (N.Y.),Rochester (N.Y.),Lewis H. Morgan,1818-11-21,1881-12-17,NaN,NaN


In [613]:
# We realize a merge between Wikidata, BnF Data and DBpedia using the BnF URIs in common
merged_final_bnf_wk_dbp = merged_final_bnf_wk.merge(final_bnf_dbp_merge, on=['id_bnf'], how="inner")
print("")
print("This method merge", len(merged_final_bnf_wk_dbp), "persons.")
print("")
cols=["id_bnf","id_wk",'id_dbp',"uri_bnf_x","uri_wk", "name_bnf_x", "dateBirth_bnf_x","dateDeath_bnf_x","placeOfBirth_bnf_x","placeOfDeath_bnf_x","bio_bnf","name_wk","dateBirth_wk","dateDeath_wk","placeOfBirth_wk","placeOfDeath_wk","name_dbp", "birthDate_dbp","deathDate_dbp","placeOfBirth_dbp","placeOfDeath_dbp"]
merged_final_bnf_wk_dbp[cols][50:100]


This method merge 58 persons.



,id_bnf,id_wk,id_dbp,uri_bnf_x,uri_wk,name_bnf_x,dateBirth_bnf_x,dateDeath_bnf_x,placeOfBirth_bnf_x,placeOfDeath_bnf_x,...,name_wk,dateBirth_wk,dateDeath_wk,placeOfBirth_wk,placeOfDeath_wk,name_dbp,birthDate_dbp,deathDate_dbp,placeOfBirth_dbp,placeOfDeath_dbp
50,9699,7500,1453,http://data.bnf.fr/ark:/12148/cb12975034z#about,http://www.wikidata.org/entity/Q1609113,Marko Veselica,1936-01-09,2017-02-17,NaN,NaN,...,Marko Veselica,1936-01-09,2017-02-17,Glavice,Zagreb,Marko Veselica,1936-01-09,2017-02-17,NaN,NaN
51,9400,1399,1914,http://data.bnf.fr/ark:/12148/cb13543155j#about,http://www.wikidata.org/entity/Q670497,Theodor Hertzka,1845-07-13,1924-10-22,Pest,"Wiebaden, Allemagne",...,Theodor Hertzka,1845-07-13,1924-10-22,Pest,Wiesbaden,Theodor Hertzka,1845-07-13,1924-10-22,Kingdom of Hungary,Weimar Republic
52,1540,53790,824,http://data.bnf.fr/ark:/12148/cb14446156k#about,http://www.wikidata.org/entity/Q64856,Eduard von Simson,1810-11-10,1899-05-02,Königsberg,Berlin,...,Eduard von Simson,1810-11-1,1899-05-02,Königsberg,Berlin,Eduard von Simson,1810-11-10,1899-05-02,Kingdom of Prussia,NaN
53,9213,1437,965,http://data.bnf.fr/ark:/12148/cb144527794#about,http://www.wikidata.org/entity/Q320857,Luigi Bodio,1840-10-12,1920-11-02,"Milan, Italie",Rome,...,Luigi Bodio,1840-10-12,1920-11-02,Milan,Rome,Luigi Bodio,1840-10-12,1920-11-02,NaN,NaN
54,9571,29858,1317,http://data.bnf.fr/ark:/12148/cb15089042m#about,http://www.wikidata.org/entity/Q13647869,Emmanuel Farhi,1978-09-08,2020-07-23,Paris (France),"Cambridge (Massachussetts, États-Unis)",...,Emmanuel Farhi,1978-09-08,2020-07-23,Paris,Cambridge,Emmanuel Farhi,1978-09-08,NaN,France,NaN
55,2274,268,1297,http://data.bnf.fr/ark:/12148/cb16153188f#about,http://www.wikidata.org/entity/Q67005,Hans Litten,1903-06-19,1938-02-05,NaN,NaN,...,Hans Litten,1903-06-19,1938-02-05,Halle (Saale),Dachau concentration camp,Hans Litten,1903-06-19,1938-02-05,German Empire,Dachau concentration camp
56,8485,5853,7282,http://data.bnf.fr/ark:/12148/cb16547873s#about,http://www.wikidata.org/entity/Q481434,Alfons Goppel,1905-10-01,1991-12-24,Ratisbonne (Allemagne),Johannesberg (Allemagne),...,Alfons Goppel,1905-10-01,1991-12-24,Reinhausen,Johannesberg,Alfons Goppel,1905-10-01,1991-12-24,Kingdom of Bavaria,Lower Franconia
57,9708,6054,725,http://data.bnf.fr/ark:/12148/cb16574250c#about,http://www.wikidata.org/entity/Q167743,Juan Álvarez y Mendizábal,1790-02-25,1853-11-03,Chiclana de la Frontera (Espagne),Madrid,...,Juan Álvarez Mendizábal,1790-02-25,1853-11-03,Chiclana de la Frontera,Madrid,Juan Álvarez Mendizábal,1790-02-25,1853-11-03,NaN,NaN


#### Match between DBpedia and Wikidata

In [45]:
indexer = recordlinkage.Index()
indexer.sortedneighbourhood(left_on='dateDeath_wk', right_on='deathDate_dbp')
candidates = indexer.index(Wikidata, DBpedia)
print(len(candidates))

10550


indexer = recordlinkage.Index()
indexer.block(left_on=['name_bnf', 'uri_bnf'],
              right_on=['name_dbp', 'uri_dbp'])
pairs = indexer.index(df_bnf, df_dbp)


candidates = indexer.index(BnF_Data, DBpedia)
print(len(candidates))

In [46]:
compare = recordlinkage.Compare()
compare.string('name_wk',
            'name_dbp',
            threshold=0.85,
            label='name_wk_dbp')
compare.exact('dateDeath_wk',
            'deathDate_dbp',
            label='deathDate_wk_dbp')
compare.exact('dateBirth_wk',
            'birthDate_dbp',
            label='birthDate_wk_dbp')
features = compare.compute(candidates, Wikidata, DBpedia)
# !!!! see if use https://recordlinkage.readthedocs.io/en/latest/ref-compare.html#recordlinkage.Compare.add works to add method like compare 'exact' on 'name'

In [47]:
features.sum(axis=1).value_counts().sort_index(ascending=False)

3.0     409
2.0     234
1.0    1110
0.0    8797
dtype: int64

In [48]:
features

,,name_wk_dbp,deathDate_wk_dbp,birthDate_wk_dbp
id_wk,id_dbp,,,
37,7331,0.0,0,0
51,1800,1.0,0,1
10006,1800,0.0,0,0
58,6668,0.0,0,0
24839,6668,0.0,0,0
...,...,...,...,...
56294,9738,0.0,0,0
56314,7415,0.0,0,0
56347,687,0.0,0,0


In [49]:
potential_matches = features[features.sum(axis=1) > 1].reset_index()
potential_matches['Score'] = potential_matches.loc[:, 'name_wk_dbp':'birthDate_wk_dbp'].sum(axis=1)
potential_matches

,id_wk,id_dbp,name_wk_dbp,deathDate_wk_dbp,birthDate_wk_dbp,Score
0,51,1800,1.0,0,1,2.0
1,152,2168,1.0,0,1,2.0
2,154,1218,1.0,0,1,2.0
3,247,4514,1.0,0,1,2.0
4,254,1671,1.0,0,1,2.0
...,...,...,...,...,...,...
638,6352,1001,1.0,0,1,2.0
639,6999,328,1.0,0,1,2.0
640,15145,4170,1.0,0,1,2.0
641,19416,5118,1.0,0,1,2.0


In [50]:
Wikidata.loc[2255,:]

uri_wk             http://www.wikidata.org/entity/Q350384
name_wk                                       Milan Jelić
dateBirth_wk                                   1956-03-26
dateDeath_wk                                    2007-09-3
placeOfBirth_wk                         Koprivna, Modriča
placeOfDeath_wk                                   Modriča
viaf                                                  NaN
uri_bnf                                               NaN
Name: 2255, dtype: object

In [51]:
DBpedia.loc[156,:]

uri_dbp                    http://dbpedia.org/resource/Boris_Fyodorov
viaf                                    http://viaf.org/viaf/91429025
name_dbp                                               Boris Fyodorov
birthDate_dbp                                              1958-02-13
deathDate_dbp                                              2008-11-20
placeOfBirth_dbp                                         Soviet Union
placeOfDeath_dbp                                               London
dbp_Name_Lookup     Boris Fyodorov|1958-02-13|2008-11-20|Soviet Un...
Name: 156, dtype: object

In [54]:
Wikidata['name_wk']=Wikidata['name_wk'].astype(str)
DBpedia['name_dbp']=DBpedia['name_dbp'].astype(str)

Wikidata['viaf']=Wikidata['viaf'].astype(str)
DBpedia['viaf']=DBpedia['viaf'].astype(str)

Wikidata['uri_wk']=Wikidata['uri_wk'].astype(str)
DBpedia['uri_dbp']=DBpedia['uri_dbp'].astype(str)

Wikidata['placeOfBirth_wk']=Wikidata['placeOfBirth_wk'].astype(str)
DBpedia['placeOfBirth_dbp']=DBpedia['placeOfBirth_dbp'].astype(str)

Wikidata['placeOfDeath_wk']=Wikidata['placeOfDeath_wk'].astype(str)
DBpedia['placeOfDeath_dbp']=DBpedia['placeOfDeath_dbp'].astype(str)

Wikidata['dateDeath_wk']=Wikidata['dateDeath_wk'].astype(str)
DBpedia['deathDate_dbp']=DBpedia['deathDate_dbp'].astype(str)

Wikidata['dateBirth_wk']=Wikidata['dateBirth_wk'].astype(str)
DBpedia['birthDate_dbp']=DBpedia['birthDate_dbp'].astype(str)

In [55]:
Wikidata['Wikidata_Name_Lookup'] = Wikidata[[
   'name_wk', 'dateBirth_wk','dateDeath_wk','placeOfBirth_wk' ,'placeOfDeath_wk' 
]].apply(lambda x: '|'.join(x), axis=1)

DBpedia['dbp_Name_Lookup'] = DBpedia[[
   'name_dbp', 'birthDate_dbp', 'deathDate_dbp','placeOfBirth_dbp', 'placeOfDeath_dbp'
]].apply(lambda x: '|'.join(x), axis=1)

Wikidata_lookup = Wikidata[['Wikidata_Name_Lookup']].reset_index()
DBpedia_lookup = DBpedia[['dbp_Name_Lookup']].reset_index()


In [56]:
Wikidata_merge = potential_matches.merge(Wikidata_lookup, how='left')

In [57]:
final_dbp_wk_merge = Wikidata_merge.merge(DBpedia_lookup, how='left')

In [58]:
cols = ['id_wk', 'id_dbp', 'Score',
        'Wikidata_Name_Lookup', 'dbp_Name_Lookup']
final_dbp_wk_merge=final_dbp_wk_merge[cols].sort_values(by=[ 'Score'], ascending=True)
print(len(final))
final[:20]

117


,id_bnf,id_dbp,Score,BnF_Name_Lookup,dbp_Name_Lookup
0,2110,9566,2.0,Mihai A. Antonescu|1907-11-18|1946-06-01|Nucet...,Mihai Antonescu|1904-11-18|1946-06-01|Kingdom ...
84,9181,3201,2.0,Vilfredo Pareto|1848-07-15|1923-08-20|Paris|Cé...,Vilfredo Pareto|1848-07-15|1923-08-19|nan|nan
34,3014,8313,2.0,Gisèle Halimi|1927-07-27|2020-07-28|La Goulett...,Gisèle Halimi|1927-07-28|2020-07-28|French pro...
33,2607,1493,2.0,William Martin Geldart|1870-06-07|1922-02-12|n...,William Martin Geldart|1870-06-07|--02-12|nan|nan
44,3539,8182,2.0,Camille Blaisot|1881-01-19|1945|Valognes (Manc...,Camille Blaisot|1881-01-19|1945-01-24|Valognes...
89,9336,4306,2.0,Jacob Marschak|1898|1977-07-27|nan|nan|Economi...,Jacob Marschak|1898-07-23|1977-07-27|Russian E...
45,3560,1976,2.0,Gyula Wlassics|1852-03-17|1937-04-30|Zalaegers...,Gyula Wlassics|1852-03-17|1937-03-30|Kingdom o...
25,1870,1054,2.0,David Josiah Brewer|1837-06-20|1910-03-27|nan|...,David Josiah Brewer|1837-06-20|1910-03-28|İzmi...
93,9470,1808,2.0,Gustav Cassel|1866-10-20|1945-01-15|Stockholm|...,Gustav Cassel|1866-10-20|1945-01-14|Stockholm|...
48,3669,5796,2.0,Vincenzo Caianiello|1932-10-02|2002-04-26|Aver...,Vincenzo Caianiello|1932-10-02|2002-04-06|Case...


----------------------------------------------------------------
### Another method

In [126]:
from sklearn.feature_extraction.text import TfidfVectorizer

Wikidata_name = Wikidata['name_wk'].unique()
vectorizer = TfidfVectorizer(min_df=1, analyzer=ngrams)
tf_idf_matrix = vectorizer.fit_transform(Wikidata_name)

BnF_Data_name = BnF_Data['name_bnf'].unique()
vectorizer = TfidfVectorizer(min_df=1, analyzer=ngrams)
tf_idf_matrix = vectorizer.fit_transform(BnF_Data_name)


In [72]:
#BnF_Data.head()

In [141]:
import re
def ngrams(string, n=3):
    string = string.encode("UTF-8", errors="ignore").decode() 
    string = string.lower()
    chars_to_remove = [')', '(', '.', '|', '[', ']', '{', '}', "'" ]
    rx = '[' + re.escape(''.join(chars_to_remove)) + ']'
    string = re.sub(rx, '', string) # remove the list of chars defined above
    string = string.replace('&', 'and')
    string = string.replace(',', ' ').replace('-', ' ')
    string = string.title() # Capital at start of each word
    string = re.sub(' +',' ',string).strip() # combine whitespace
    string = ' ' + string + ' ' # pad
    string = re.sub(r'[,-./]|\sBD', r'', string)
    ngrams = zip(*[string[i:] for i in range(n)])
    return [''.join(ngram) for ngram in ngrams]

In [142]:
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer

In [143]:
BnF_Data_clean = BnF_Data["name_bnf"].unique()

print('Vectorizing the data - this could take a few minutes for large datasets...')
vectorizer = TfidfVectorizer(min_df=1, analyzer=ngrams, lowercase=False)

tfidf = vectorizer.fit_transform(BnF_Data_clean)
print('Vectorizing completed...')

from sklearn.neighbors import NearestNeighbors
nbrs = NearestNeighbors(n_neighbors=1, n_jobs=-1).fit(tfidf)

name_column = 'name_wk' #column to match against in the messy data
unique_name = set(Wikidata[name_column].values) # set used for increased performance

Vectorizing the data - this could take a few minutes for large datasets...
Vectorizing completed...


In [144]:
###matching query:
def getNearestN(query):
    queryTFIDF_ = vectorizer.transform(query)
    distances, indices = nbrs.kneighbors(queryTFIDF_)
    return distances, indices

import time
t1 = time.time()
print('getting nearest n...')
distances, indices = getNearestN(unique_name)
t = time.time()-t1
print("COMPLETED IN:", t)

unique_name = list(unique_name) #need to convert back to a list
print('finding matches...')
matches = []
for i,j in enumerate(indices):
    temp = [round(distances[i][0],2), BnF_Data.values[j][0][2],unique_name[i]]
    matches.append(temp)

print('Building data frame...')  
matches = pd.DataFrame(matches, columns=['Match confidence (lower is better)','Matched name','Original name'])
print('Done')

getting nearest n...


KeyboardInterrupt: 

In [151]:
# Match string with TF_IDF method
# https://bergvca.github.io/2017/10/14/super-fast-string-matching.html
import numpy as np
from scipy.sparse import csr_matrix
from scipy.sparse import rand
import sparse_dot_topn.sparse_dot_topn as ct
from sparse_dot_topn import awesome_cossim_topn

N = 10
a = rand(100, 1000000, density=0.005, format='csr')
b = rand(1000000, 200, density=0.005, format='csr')

# Use standard implementation

c = awesome_cossim_topn(a, b, N, 0.01)

# Use parallel implementation with 4 threads

d = awesome_cossim_topn(a, b, N, 0.01, use_threads=True, n_jobs=4)

In [152]:
import time
t1 = time.time()
matches = awesome_cossim_top(tf_idf_matrix, tf_idf_matrix.transpose(), 10, 0.8)
t = time.time()-t1
print("SELFTIMED:", t)

SELFTIMED: 0.3138604164123535


In [153]:
def get_matches_df(sparse_matrix, name_vector, top=100):
    non_zeros = sparse_matrix.nonzero()
    
    sparserows = non_zeros[0]
    sparsecols = non_zeros[1]
    
    if top:
        nr_matches = top
    else:
        nr_matches = sparsecols.size
    
    left_side = np.empty([nr_matches], dtype=object)
    right_side = np.empty([nr_matches], dtype=object)
    similairity = np.zeros(nr_matches)
    
    for index in range(0, nr_matches):
        left_side[index] = name_vector[sparserows[index]]
        right_side[index] = name_vector[sparsecols[index]]
        similairity[index] = sparse_matrix.data[index]
    
    return pd.DataFrame({'left_side': left_side,
                            'right_side': right_side,
                              'similairity': similairity})
        


In [156]:
matches_df = get_matches_df(matches, BnF_Data_name, top=11171)
matches_df = matches_df[matches_df['similairity'] < 0.99999] # Remove all exact matches
matches_df.sample(20)


,left_side,right_side,similairity
9358,Michel Morineau,Michel Morin,0.820119
6672,Jean Schmidt,Jean Charles Schmidt,0.806333
9999,Jean Charles Schmidt,Jean Schmidt,0.806333
1860,Georges-Henri Bousquet,Georges Bousquet,0.820439
3491,ʿAlī ʿAlāʾ al-Dīn al- ʿĀlūsī,Ṣalāḥ al-Dīn al- Nāhī,0.804265
6364,Édouard Grar,Édouard Gérard,0.921296
5807,Raymond de Geouffre de La Pradelle,Paul de Geouffre de La Pradelle,0.836728
6209,Charles Houyvet,Henri Charles Houyvet,0.890240
1463,Hippolyte Bérard Des Glajeux,Étienne-Hippolyte-Paul Bérard des Glajeux,0.850011
7435,Charles-Jules Giraud,Charles Giraud,0.867012


In [135]:
matches_df.sort_values(['similairity'], ascending=False).head(10)

,left_side,right_side,similairity
4268,Joseph-Marie de Kersauson de Pennendreff,Joseph-Marc-Marie de Kersauson de Pennendreff,0.948466
5851,Joseph-Marc-Marie de Kersauson de Pennendreff,Joseph-Marie de Kersauson de Pennendreff,0.948466
6809,Rodolphe Dareste de La Chavanne,Pierre-Rodolphe Dareste de La Chavanne,0.937893
1387,Pierre-Rodolphe Dareste de La Chavanne,Rodolphe Dareste de La Chavanne,0.937893
6053,Charles Merveilleux du Vignaux,François-Charles Merveilleux du Vignaux,0.935526
5884,François-Charles Merveilleux du Vignaux,Charles Merveilleux du Vignaux,0.935526
4995,Jean Gilbert Villeneuve,Gilbert Villeneuve,0.925843
910,Gilbert Villeneuve,Jean Gilbert Villeneuve,0.925843
7108,Édouard Gérard,Édouard Grar,0.921296
6364,Édouard Grar,Édouard Gérard,0.921296


In [86]:
matches.head(10)

,Match confidence (lower is better),Matched name,Original name
0,1.08,Sándor Boschan,Ernst Geiger
1,1.03,Armand Dorville,Robert Basmann
2,1.10,Jesse James,Paul Javor
3,1.00,Jean-Claude Colliard,Luis Bates
4,1.04,Gian Domenico Pisapia,Max Ludwig Boeckh
5,1.12,Charles Libman,Yann Moulier-Boutang
6,1.15,Frank Eaton,Haldur Grüner
7,1.20,Gabriele Criscuoli,Mikalai Zaichanka
8,0.99,József Katona,Bonifaz Sander
9,1.13,Nicolás Tenorio y Cerero,Angel Agache


### Between Wikidata and DBpedia

In [615]:
merged_df_wk_dbp = pd.merge( Wikidata, DBpedia , on='viaf', how='inner', sort='viaf')
merged_df_wk_dbp[:10]

,uri_wk,name_wk,dateBirth_wk,dateDeath_wk,placeOfBirth_wk,placeOfDeath_wk,viaf,uri_bnf,Wikidata_Name_Lookup,uri_dbp,name_dbp,birthDate_dbp,deathDate_dbp,placeOfBirth_dbp,placeOfDeath_dbp
0,http://www.wikidata.org/entity/Q887720,José Serrato,1868-09-3,1960-09-07,Montevideo,Montevideo,http://viaf.org/viaf/100069209,http://data.bnf.fr/ark:/12148/cb169866051#about,José Serrato|1868-09-3|1960-09-07|Montevideo|M...,http://dbpedia.org/resource/José_Serrato,José Serrato,1868-09-30,1960-09-07,Uruguay,Uruguay
1,http://www.wikidata.org/entity/Q9387,Max Weber,1864-04-21,1920-06-14,Erfurt,Munich,http://viaf.org/viaf/100180950,http://data.bnf.fr/ark:/12148/cb11928969n#about,Max Weber|1864-04-21|1920-06-14|Erfurt|Munich,http://dbpedia.org/resource/Max_Weber,NaN,1864-04-21,1920-06-14,Erfurt,Weimar Republic
2,http://www.wikidata.org/entity/Q510386,José Hermano Saraiva,1919-10-03,2012-07-2,Leiria,Setúbal,http://viaf.org/viaf/100238197,http://data.bnf.fr/ark:/12148/cb12396660s#about,José Hermano Saraiva|1919-10-03|2012-07-2|Leir...,http://dbpedia.org/resource/José_Hermano_Saraiva,José Hermano Saraiva,1919-10-03,2012-07-20,First Portuguese Republic,Palmela
3,http://www.wikidata.org/entity/Q7347192,Robert M. Townsend,1948-04-23,nan,Cambridge,nan,http://viaf.org/viaf/100284933,http://data.bnf.fr/ark:/12148/cb12280684z#about,Robert M. Townsend|1948-04-23|nan|Cambridge|nan,http://dbpedia.org/resource/Robert_M._Townsend,Robert M. Townsend,1948-04-23,NaN,United States,NaN
4,http://www.wikidata.org/entity/Q5785982,Carl B. Hamilton,1946-01-01,nan,Gothenburg,nan,http://viaf.org/viaf/100285708,http://data.bnf.fr/ark:/12148/cb12291672d#about,Carl B. Hamilton|1946-01-01|nan|Gothenburg|nan,http://dbpedia.org/resource/Carl_B._Hamilton,Carl B. Hamilton,1946-01-01,NaN,NaN,NaN
5,http://www.wikidata.org/entity/Q437403,John Peters Humphrey,1905-04-3,1995-03-14,New Brunswick,Montreal,http://viaf.org/viaf/100966624,http://data.bnf.fr/ark:/12148/cb122145877#about,John Peters Humphrey|1905-04-3|1995-03-14|New ...,http://dbpedia.org/resource/John_Peters_Humphrey,John Peters Humphrey,1905-04-30,1995-03-14,NaN,NaN
6,http://www.wikidata.org/entity/Q3525487,Thomas Porcher,1977-07-05,nan,Drancy,nan,http://viaf.org/viaf/101006249,http://data.bnf.fr/ark:/12148/cb16141444z#about,Thomas Porcher|1977-07-05|nan|Drancy|nan,http://dbpedia.org/resource/Thomas_Porcher,Thomas Porcher,1977-07-05,NaN,NaN,NaN
7,http://www.wikidata.org/entity/Q505520,Kenneth E. Boulding,1910-01-18,1993-03-18,Liverpool,Boulder,http://viaf.org/viaf/101364972,http://data.bnf.fr/ark:/12148/cb120338616#about,Kenneth E. Boulding|1910-01-18|1993-03-18|Live...,http://dbpedia.org/resource/Kenneth_E._Boulding,K. E. Boulding,1910-01-18,1993-03-18,United Kingdom,"Boulder, Colorado"
8,http://www.wikidata.org/entity/Q19864781,Rania Antonopoulos,1960-12-17,nan,Athens,nan,http://viaf.org/viaf/101406479,http://data.bnf.fr/ark:/12148/cb16511866f#about,Rania Antonopoulos|1960-12-17|nan|Athens|nan,http://dbpedia.org/resource/Rania_Antonopoulou,Rania Antonopoulou,1960-12-17,NaN,Greece,NaN
9,http://www.wikidata.org/entity/Q364315,Jacques Attali,1943-11-01,nan,Algiers,nan,http://viaf.org/viaf/101721040,http://data.bnf.fr/ark:/12148/cb11889358n#about,Jacques Attali|1943-11-01|nan|Algiers|nan,http://dbpedia.org/resource/Jacques_Attali,Jacques Attali,1943-11-01,NaN,Algiers,NaN


In [617]:
print("the number of merged data from DBpedia and Wikidata is ",len(merged_df_wk_dbp), "rows.")
print("")
print("The proportion of the number of merged data from DBpedia with Wikidata is ",((len(merged_df_wk_dbp))/(len(DBpedia))*100),"%")
print("")
print("proportion of the number of merged data from Wikidata with DBpedia is ",((len(merged_df_wk_dbp))/(len(Wikidata))*100),"%")

the number of merged data from DBpedia and Wikidata is  1254 rows.

The proportion of the number of merged data from DBpedia with Wikidata is  28.61054072553046 %

proportion of the number of merged data from Wikidata with DBpedia is  2.3520144046815217 %


### Between Wikidata and BnF Data

In [38]:
merged_df_2 = pd.merge( df_wk, df_bnf , on='viaf', how='inner', sort='viaf')
print(len(merged_df_2))
merged_df_2[:10]

112


,uri_wk,viaf,name_wk,year_wk,uri_bnf,name_bnf,sName,year_bnf,bio_bnf
0,http://www.wikidata.org/entity/Q540253,http://viaf.org/viaf/100277874,Clemens Maria Franz von Bönninghausen,1785,http://data.bnf.fr/ark:/12148/cb165146162#about,Clemens Maria Franz von Bönninghausen,,1785,"Avocat, agriculteur, botaniste, homéopathe. - ..."
1,http://www.wikidata.org/entity/Q11724800,http://viaf.org/viaf/101647977,Jerzy Karol Kurnatowski,1874,http://data.bnf.fr/ark:/12148/cb10528392p#about,Jerzy Kurnatowski,,1874,"Publiciste, juriste et économiste"
2,http://www.wikidata.org/entity/Q11738367,http://viaf.org/viaf/101863288,Kazimierz Studentowicz,1903,http://data.bnf.fr/ark:/12148/cb11261387v#about,Kazimierz Studentowicz,,1903,"Juriste, homme politique"
3,http://www.wikidata.org/entity/Q104820757,http://viaf.org/viaf/107036313,Félix Garcin,1879,http://data.bnf.fr/ark:/12148/cb13073916m#about,Félix Garcin,,1879,"Journaliste, directeur de ""Nouvelliste"", Lyon...."
4,http://www.wikidata.org/entity/Q1680590,http://viaf.org/viaf/107145857823423020439,Josef Redlich,1869,http://data.bnf.fr/ark:/12148/cb12818732h#about,Josef Redlich,,1869,Professeur de droit public et d'administration...
5,http://www.wikidata.org/entity/Q518859,http://viaf.org/viaf/108188941,Gordon Tullock,1922,http://data.bnf.fr/ark:/12148/cb11927239j#about,Gordon Tullock,,1922,"Docteur en droit (University of Chicago, Ill.,..."
6,http://www.wikidata.org/entity/Q88911,http://viaf.org/viaf/108482851,Otto Nathan,1893,http://data.bnf.fr/ark:/12148/cb12874301d#about,Otto Nathan,,1893,Économiste. - Avocat. - Exécuteur testamentair...
7,http://www.wikidata.org/entity/Q652154,http://viaf.org/viaf/108587991,Alexis Jacquemin,1938,http://data.bnf.fr/ark:/12148/cb119084288#about,Alexis Jacquemin,,1938,Juriste et économiste. - Professeur à l'Univer...
8,http://www.wikidata.org/entity/Q87110924,http://viaf.org/viaf/110494713,Henri Denis,1913,http://data.bnf.fr/ark:/12148/cb12103181f#about,Henri Denis,,1913,"Docteur en droit (Paris, 1938). - Professeur d..."
9,http://www.wikidata.org/entity/Q1345621,http://viaf.org/viaf/111314151,Marco Biagi,1950,http://data.bnf.fr/ark:/12148/cb150888041#about,Marco Biagi,,1950,Juriste


In [39]:
print("The number of merged data from BnF Data and Wikidata is ",len(merged_df_2), "rows.")

print("")

print("The proportion of the number of merged data from BnF Data with Wikidata is ",((len(merged_df_2))/(len(df_bnf))*100),"%")

print("")

print("The proportion of the number of merged data from Wikidata with Wikidata is ",((len(merged_df_2))/(len(df_wk))*100),"%")

The number of merged data from BnF Data and Wikidata is  112 rows.

The proportion of the number of merged data from BnF Data with Wikidata is  1.224445173280857 %

The proportion of the number of merged data from Wikidata with Wikidata is  0.5179669796050502 %


### Between DBpedia and BnF Data

In [40]:
merged_df_3 = pd.merge( df_bnf, df_dbp , on='viaf', how='inner', sort='viaf')
print(len(merged_df_3))
merged_df_3[:10]

88


,uri_bnf,viaf,name_bnf,sName,year_bnf,bio_bnf,uri_dbp,name_dbp,year_dbp
0,http://data.bnf.fr/ark:/12148/cb122145877#about,http://viaf.org/viaf/100966624,John Humphrey,,1905,Juriste. - A été professeur de droit internati...,http://dbpedia.org/resource/John_Peters_Humphrey,John Peters Humphrey,1905
1,http://data.bnf.fr/ark:/12148/cb12327654n#about,http://viaf.org/viaf/107536763,Louis Renault,,1843,Juriste. - Professeur de droit international à...,http://dbpedia.org/resource/Louis_Renault_(jur...,Louis Renault,1843
2,http://data.bnf.fr/ark:/12148/cb122775427#about,http://viaf.org/viaf/108173876,Ronald Myles Dworkin,,1931,Juriste. - Professeur de jurisprudence à la Ya...,http://dbpedia.org/resource/Ronald_Dworkin,,1931
3,http://data.bnf.fr/ark:/12148/cb11927239j#about,http://viaf.org/viaf/108188941,Gordon Tullock,,1922,"Docteur en droit (University of Chicago, Ill.,...",http://dbpedia.org/resource/Gordon_Tullock,Gordon Tullock,1922
4,http://data.bnf.fr/ark:/12148/cb120906270#about,http://viaf.org/viaf/108565309,Paul Abraham Freund,,1908,"Professeur de droit, ""Harvard Law School""",http://dbpedia.org/resource/Paul_A._Freund,Paul Abraham Freund,1908
5,http://data.bnf.fr/ark:/12148/cb119084288#about,http://viaf.org/viaf/108587991,Alexis Jacquemin,,1938,Juriste et économiste. - Professeur à l'Univer...,http://dbpedia.org/resource/Alexis_Jacquemin,Alexis Jacquemin,1938
6,http://data.bnf.fr/ark:/12148/cb128832222#about,http://viaf.org/viaf/108624624,Muḥammad Ẓafr Allāh H̱ān,,1893,"Juriste, diplomate et homme politique",http://dbpedia.org/resource/Muhammad_Zafarulla...,CH Muhammad Zafarullah Khan,1893
7,http://data.bnf.fr/ark:/12148/cb12299375j#about,http://viaf.org/viaf/108794549,Karl Engisch,,1899,Juriste. - Spécialiste de philosophie du droit...,http://dbpedia.org/resource/Karl_Engisch,Karl Engisch,1899
8,http://data.bnf.fr/ark:/12148/cb118935370#about,http://viaf.org/viaf/111389197,Georges Bousquet,,1846,Avocat au Barreau de Paris (en 1866). - Engagé...,http://dbpedia.org/resource/Georges_Hilaire_Bo...,Georges Hilaire Bousquet,1845
9,http://data.bnf.fr/ark:/12148/cb12328362p#about,http://viaf.org/viaf/11396531,John Paul Stevens,,1920,Juriste américain,http://dbpedia.org/resource/John_Paul_Stevens,John Paul Stevens,1920


In [41]:
print("The number of merged data from BnF Data and Wikidata is ",len(merged_df_3), "rows.")

print("")

print("The proportion of the number of merged data from BnF Data with DBpedia is ",((len(merged_df_3))/(len(df_dbp))*100),"%")

print("")

print("The proportion of the number of merged data from DBpedia with BnF Data is ",((len(merged_df_3))/(len(df_bnf))*100),"%")

The number of merged data from BnF Data and Wikidata is  88 rows.

The proportion of the number of merged data from BnF Data with DBpedia is  5.333333333333334 %

The proportion of the number of merged data from DBpedia with BnF Data is  0.9620640647206734 %


### Between Wikidata, BnF Data and DBpedia

In [42]:
merged_df = pd.merge( merged_df_1, df_bnf , on='viaf', how='inner', sort='viaf')
merged_df[:10]

,uri_wk,viaf,name_wk,year_wk,uri_dbp,name_dbp,year_dbp,uri_bnf,name_bnf,sName,year_bnf,bio_bnf
0,http://www.wikidata.org/entity/Q518859,http://viaf.org/viaf/108188941,Gordon Tullock,1922,http://dbpedia.org/resource/Gordon_Tullock,Gordon Tullock,1922,http://data.bnf.fr/ark:/12148/cb11927239j#about,Gordon Tullock,,1922,"Docteur en droit (University of Chicago, Ill.,..."
1,http://www.wikidata.org/entity/Q652154,http://viaf.org/viaf/108587991,Alexis Jacquemin,1938,http://dbpedia.org/resource/Alexis_Jacquemin,Alexis Jacquemin,1938,http://data.bnf.fr/ark:/12148/cb119084288#about,Alexis Jacquemin,,1938,Juriste et économiste. - Professeur à l'Univer...
2,http://www.wikidata.org/entity/Q3085838,http://viaf.org/viaf/32062931,François Simiand,1873,http://dbpedia.org/resource/François_Simiand,François Simiand,1873,http://data.bnf.fr/ark:/12148/cb12301152q#about,François Simiand,,1873,Philosophe. - Agrégé de philosophie. - Docteur...
3,http://www.wikidata.org/entity/Q61956,http://viaf.org/viaf/44308789,Lorenz von Stein,1815,http://dbpedia.org/resource/Lorenz_von_Stein,Lorenz von Stein,1815,http://data.bnf.fr/ark:/12148/cb12001622n#about,Lorenz von Stein,,1815,"Juriste et économiste. - Professeur à Kiel, Al..."
4,http://www.wikidata.org/entity/Q231690,http://viaf.org/viaf/44331988,B. R. Ambedkar,1891,http://dbpedia.org/resource/B._R._Ambedkar,Bhimrao Ramji Ambedkar,1891,http://data.bnf.fr/ark:/12148/cb12126992f#about,Bhimrao Ramji Ambedkar,,1891,Homme politique d'origine harijan mahar. - Étu...
5,http://www.wikidata.org/entity/Q215961,http://viaf.org/viaf/50021033,Franz Hermann Schulze-Delitzsch,1808,http://dbpedia.org/resource/Franz_Hermann_Schu...,Hermann Schulze-Delitzsch,1808,http://data.bnf.fr/ark:/12148/cb12088660j#about,Hermann Schulze-Delitzsch,,1808,"Juriste, homme politique et économiste alleman..."
6,http://www.wikidata.org/entity/Q4893263,http://viaf.org/viaf/69263532,Joan Sardà i Dexeus,1910,http://dbpedia.org/resource/Joan_Sardà_i_Dexeus,Joan Sardà i Dexeus,1910,http://data.bnf.fr/ark:/12148/cb158098327#about,Juan Sardá Dexeus,,1910,Docteur en droit. - Économiste
7,http://www.wikidata.org/entity/Q7836141,http://viaf.org/viaf/73921034,Travers Twiss,1809,http://dbpedia.org/resource/Travers_Twiss,Travers Twiss,1809,http://data.bnf.fr/ark:/12148/cb12314495r#about,Travers Twiss,,1809,Juriste. - Spécialiste de droit international


In [43]:
print("The number of merged data from DBpedia, Wikidata and BnF Data is",len(merged_df),"rows.")
print("")
print("The proportion of the number of merged data from DBpedia, Wikidata and BnF Data is ",(len(merged_df))/(len(df_bnf))*100,"%" )

The number of merged data from DBpedia, Wikidata and BnF Data is 8 rows.

The proportion of the number of merged data from DBpedia, Wikidata and BnF Data is  0.08746036952006123 %


In [42]:
df1 = pd.DataFrame({'user_id': ['id001', 'id002', 'id003', 'id004', 'id005', 'id006', 'id007'],
                    'first_name': ['Rivi', 'Wynnie', 'Kristos', 'Madalyn', 'Tobe', 'Regan', 'Kristin'],
                    'last_name': ['Valti', 'McMurty', 'Ivanets', 'Max', 'Riddich', 'Huyghe', 'Illis'],
                    'email': ['rvalti0@example.com', 'wmcmurty1@example.com', 'kivanets2@example.com',
                              'mmax3@example.com', 'triddich4@example.com', 'rhuyghe@example.com', 'killis4@example.com']
                    })

In [43]:
df2 = pd.DataFrame({'user_id': ['id001', 'id002', 'id003', 'id004', 'id005'],
                    'image_url': ['http://example.com/img/id001.png', 'http://example.com/img/id002.jpg',
                                  'http://example.com/img/id003.bmp', 'http://example.com/img/id004.jpg',
                                  'http://example.com/img/id005.png']
                    })

In [11]:
df3_merged = pd.merge(df1, df2)
df3_merged 

,user_id,first_name,last_name,email,image_url
0,id001,Rivi,Valti,rvalti0@example.com,http://example.com/img/id001.png
1,id002,Wynnie,McMurty,wmcmurty1@example.com,http://example.com/img/id002.jpg
2,id003,Kristos,Ivanets,kivanets2@example.com,http://example.com/img/id003.bmp
3,id004,Madalyn,Max,mmax3@example.com,http://example.com/img/id004.jpg
4,id005,Tobe,Riddich,triddich4@example.com,http://example.com/img/id005.png
